In [6]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from datetime import datetime
import time
import numpy as np
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException

# --- Scraping Functions ---
def wait_for_element(driver, locator):
    return WebDriverWait(driver, 10).until(EC.element_to_be_clickable(locator))

def select_date_month_day(driver, date_str, date_input_id):
    date_to_select = datetime.strptime(date_str, '%Y-%m-%d')
    date_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, date_input_id)))
    date_input.click()
    month_select = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-datepicker-month')))
    month_option = month_select.find_element(By.XPATH, f"//option[@value='{date_to_select.month - 1}']")
    month_option.click()
    day = date_to_select.day
    day_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//td[@data-handler='selectDay']/a[text()='{day}']")))
    day_element.click()
    time.sleep(1)

def select_date(driver, date_str, date_input_id):
    date_to_select = datetime.strptime(date_str, '%Y-%m-%d')
    date_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, date_input_id)))
    date_input.click()
    month_select = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-datepicker-month')))
    month_option = month_select.find_element(By.XPATH, f"//option[@value='{date_to_select.month - 1}']")
    month_option.click()
    year_select = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-datepicker-year')))
    year_option = year_select.find_element(By.XPATH, f"//option[@value='{date_to_select.year}']")
    year_option.click()
    day = date_to_select.day
    day_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//td[@data-handler='selectDay']/a[text()='{day}']")))
    day_element.click()
    time.sleep(1)

def extract_grid_data_clm_summary(driver):
    data = []
    try:
        total_pages_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "sp_1_pjqgridClmSummbyProv")))
        total_pages = int(total_pages_element.text.strip())
    except:
        return data
    for current_page in range(1, total_pages + 1):
        time.sleep(3)
        driver.execute_script("window.scrollTo(0, 0);")
        grid = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "jqgridClmSummbyProv")))
        rows = WebDriverWait(grid, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "tr.jqgrow")))
        for row in rows:
            try:
                provider_name = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_ProvName']").text
                visits = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_NoOfVisit']").text
                claim = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_ClmAmt']").text
                try:
                    total_mc = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_TotalMC']").text
                except:
                    total_mc = '0'  # Default to 0 if Total MC column is missing
                data.append({'Provider Name': provider_name, 'No of Visits': visits, 'Total Claim': claim, 'Total MC (Days)': total_mc})
            except Exception as e:
                print(f"Error extracting row: {e}")
                continue
        if current_page < total_pages:
            try:
                next_button_div = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn.btn-sm.btn-default span.fa.fa-forward")))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button_div)
                next_button_div.click()
                WebDriverWait(driver, 10).until(EC.staleness_of(rows[0]))
            except:
                break
    print(f"Scraped claim data: {data[:5]}")  # Log first 5 rows for debugging
    return data

def extract_grid_data_patient_analysis(driver):
    all_data = []
    while True:
        driver.execute_script("window.scrollTo(0, 0);")
        grid = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "jqgridCorpMcAnalysis")))
        rows = grid.find_elements(By.CSS_SELECTOR, "tr.jqgrow")
        for row in rows:
            try:
                employee_name = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_MEM_NAME']").text
                employee_no = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_MEM_EMPID']").text
                division = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_MEM_EMPDIVISION']").text
                total_visit = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalVisit']").text
                total_mc = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalMC']").text
                total_claim_own = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalClaim_Own']").text
                total_claim_dep = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalClaim_Dep']").text
                all_data.append({
                    'Employee Name': employee_name, 'Employee No': employee_no, 'Division/Department': division,
                    'Total Visit': total_visit, 'Total MC (Days)': total_mc, 'Total Claim (Own)': total_claim_own,
                    'Total Claim (Dep)': total_claim_dep
                })
            except:
                continue
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            next_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn.btn-sm.btn-default span.fa.fa-forward")))
            parent_div = next_button.find_element(By.XPATH, "./parent::div")
            if "disabled" in parent_div.get_attribute("class"):
                break
            driver.execute_script("arguments[0].scrollIntoView(true);", parent_div)
            parent_div.click()
            WebDriverWait(driver, 10).until(EC.staleness_of(rows[0]))
        except:
            break
    return all_data

# --- Data Scraping Function ---
def scrape_data(url, user_id, password, start_date_patient, end_date_patient, start_date_claim, end_date_claim):
    edge_options = Options()
    driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()), options=edge_options)
    try:
        driver.get(url)
        image = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//img[@src='/ClaimEXMVR/Servlet_LoadImage?SFC=loadImage&imageName=icorporate.png']")))
        image.click()
        user_id_field = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.NAME, "txtloginid")))
        user_id_field.send_keys(user_id)
        password_field = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "inputpss")))
        password_field.send_keys(password)
        sign_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn.btn-primary[type='submit']")))
        sign_button.click()
        continue_button = wait_for_element(driver, (By.XPATH, "//button[text()='Continue']"))
        continue_button.click()

        productivity_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[span[text()='Productivity Reports']]")))
        productivity_link.click()
        patient_analysis_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Patient_Analysis_Report'][span[text()=' Patient Analysis Report ']]")))
        patient_analysis_link.click()
        select_date(driver, start_date_patient, "txtStartDate")
        select_date(driver, end_date_patient, "txtEndDate")
        search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
        search_button.click()
        time.sleep(5)
        dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
        driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "select.ui-pg-selbox")))
        select = Select(dropdown)
        select.select_by_value("100")
        time.sleep(10)
        patient_data = extract_grid_data_patient_analysis(driver)
        patient_df = pd.DataFrame(patient_data)

        reg_claims_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[.//span[contains(text(), 'Registration') and contains(text(), 'Claims')]]")))
        reg_claims_link.click()
        providers_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Claim_Summary_by_Provider_Analysis'][span[text()=' Claim Summary by Providers ']]")))
        providers_link.click()
        time.sleep(5)
        select_date_month_day(driver, start_date_claim, "txtFromDate")
        select_date_month_day(driver, end_date_claim, "txtToDate")
        time.sleep(2)
        search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
        driver.execute_script("arguments[0].click();", search_button)
        time.sleep(10)
        dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
        driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "select.ui-pg-selbox")))
        select = Select(dropdown)
        select.select_by_value("100")
        time.sleep(5)
        claim_data = extract_grid_data_clm_summary(driver)
        claim_df = pd.DataFrame(claim_data)

        productivity1_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[span[text()='Productivity Reports']]")))
        productivity1_link.click()
        mc_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/MC_HealthCare_By_Provider'][span[text()=' MC by Provider ']]")))
        mc_link.click()
        select_date(driver, start_date_patient, "txtStartDate")
        select_date(driver, end_date_patient, "txtEndDate")
        search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
        search_button.click()
        time.sleep(5)
        dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
        driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "select.ui-pg-selbox")))
        select = Select(dropdown)
        select.select_by_value("100")
        time.sleep(10)
        mc_data = extract_grid_data_mc(driver)
        mc_df = pd.DataFrame(claim_data)

        # Convert to numeric and calculate derived columns
        numeric_cols_patient = ['Total Visit', 'Total MC (Days)', 'Total Claim (Own)', 'Total Claim (Dep)']
        for col in numeric_cols_patient:
            patient_df[col] = pd.to_numeric(patient_df[col], errors='coerce')
        patient_df['Total Claim (Combined)'] = patient_df['Total Claim (Own)'] + patient_df['Total Claim (Dep)']
        patient_df['Avg Claim per Visit'] = patient_df['Total Claim (Combined)'] / patient_df['Total Visit']

        numeric_cols_claim = ['No of Visits', 'Total Claim', 'Total MC (Days)']
        for col in numeric_cols_claim:
            claim_df[col] = pd.to_numeric(claim_df[col], errors='coerce')
        claim_df['Avg Claim per Visit'] = claim_df['Total Claim'] / claim_df['No of Visits']

        print(f"claim_df columns: {claim_df.columns.tolist()}")  # Debug column names
        print(f"claim_df head: {claim_df.head()}")  # Debug first few rows

        return patient_df, claim_df, "Data scraped successfully!"
    except Exception as e:
        return None, None, f"Error: {str(e)}"
    finally:
        driver.quit()

# --- Plotting Function ---
def generate_selected_plot(chart_type, patient_df, claim_df):
    if claim_df is None or patient_df is None:
        return None
    
    sns.set(style="whitegrid")
    plt.figure(figsize=(12, 6))
    ax = plt.gca()

    try:
        if chart_type == "Top 10 Providers by Avg Claim per Visit":
            top_providers = claim_df.sort_values(by='Avg Claim per Visit', ascending=False).head(10)
            sns.barplot(data=top_providers, x='Avg Claim per Visit', y='Provider Name', hue='Provider Name', palette='viridis', legend=False, ax=ax)
            plt.title('Top 10 Providers by Average Claim per Visit')
            for i, v in enumerate(top_providers['Avg Claim per Visit']):
                ax.text(v + 0.02 * max(top_providers['Avg Claim per Visit'], default=0), i, f'{v:.2f}', va='center', ha='left', color='black', fontsize=10)

        elif chart_type == "Top 10 Employees by Avg Claim per Visit":
            top_employees = patient_df.sort_values(by='Avg Claim per Visit', ascending=False).head(10)
            sns.barplot(data=top_employees, x='Avg Claim per Visit', y='Employee Name', hue='Employee Name', palette='magma', legend=False, ax=ax)
            plt.title('Top 10 Employees by Average Claim per Visit')
            for i, v in enumerate(top_employees['Avg Claim per Visit']):
                ax.text(v + 0.02 * max(top_employees['Avg Claim per Visit'], default=0), i, f'{v:.2f}', va='center', ha='left', color='black', fontsize=10)

        elif chart_type == "Correlation Matrix":
            corr = patient_df[['Total Visit', 'Total MC (Days)', 'Total Claim (Own)', 'Total Claim (Dep)', 'Total Claim (Combined)', 'Avg Claim per Visit']].corr()
            plt.figure(figsize=(10, 8))
            sns.heatmap(corr, annot=True, cmap='coolwarm', linewidths=0.5, fmt='.2f')
            plt.title('Correlation Matrix')

        elif chart_type == "Top 10 Employees by Total Visit":
            top_employees_visit = patient_df.sort_values(by='Total Visit', ascending=False).head(10)
            sns.barplot(data=top_employees_visit, x='Total Visit', y='Employee Name', hue='Employee Name', palette='Blues_d', legend=False, ax=ax)
            plt.title('Top 10 Employees by Total Visit')
            for i, v in enumerate(top_employees_visit['Total Visit']):
                ax.text(v + 0.02 * max(top_employees_visit['Total Visit'], default=0), i, f'{v}', va='center', ha='left', color='black', fontsize=10)

        elif chart_type == "Top 10 Employees by Total Claim":
            top_employees_claim = patient_df.sort_values(by='Total Claim (Combined)', ascending=False).head(10)
            sns.barplot(data=top_employees_claim, x='Total Claim (Combined)', y='Employee Name', hue='Employee Name', palette='Reds_d', legend=False, ax=ax)
            plt.title('Top 10 Employees by Total Claim')
            for i, v in enumerate(top_employees_claim['Total Claim (Combined)']):
                ax.text(v + 0.02 * max(top_employees_claim['Total Claim (Combined)'], default=0), i, f'{v:.2f}', va='center', ha='left', color='black', fontsize=10)

        elif chart_type == "Top 10 Employees by Total MC (Days)":
            top_employees_mc = patient_df.sort_values(by='Total MC (Days)', ascending=False).head(10)
            sns.barplot(data=top_employees_mc, x='Total MC (Days)', y='Employee Name', hue='Employee Name', palette='Greens_d', legend=False, ax=ax)
            plt.title('Top 10 Employees by Total MC (Days)')
            for i, v in enumerate(top_employees_mc['Total MC (Days)']):
                ax.text(v + 0.02 * max(top_employees_mc['Total MC (Days)'], default=0), i, f'{v}', va='center', ha='left', color='black', fontsize=10)

        elif chart_type == "Top 10 Providers by Total Visit":
            if 'No of Visits' not in claim_df.columns:
                raise ValueError("Column 'No of Visits' not found in claim_df")
            top_providers_visit = claim_df.sort_values(by='No of Visits', ascending=False).head(10)
            sns.barplot(data=top_providers_visit, x='No of Visits', y='Provider Name', hue='Provider Name', palette='Purples_d', legend=False, ax=ax)
            plt.title('Top 10 Providers by Total Visit')
            for i, v in enumerate(top_providers_visit['No of Visits']):
                ax.text(v + 0.02 * max(top_providers_visit['No of Visits'], default=0), i, f'{v}', va='center', ha='left', color='black', fontsize=10)

        elif chart_type == "Top 10 Providers by Total Claim":
            top_providers_claim = claim_df.sort_values(by='Total Claim', ascending=False).head(10)
            sns.barplot(data=top_providers_claim, x='Total Claim', y='Provider Name', hue='Provider Name', palette='Oranges_d', legend=False, ax=ax)
            plt.title('Top 10 Providers by Total Claim')
            for i, v in enumerate(top_providers_claim['Total Claim']):
                ax.text(v + 0.02 * max(top_providers_claim['Total Claim'], default=0), i, f'{v:.2f}', va='center', ha='left', color='black', fontsize=10)

        elif chart_type == "Top 10 Providers by Total MC (Days)":
            top_providers_mc = claim_df.sort_values(by='Total MC (Days)', ascending=False).head(10)
            sns.barplot(data=top_providers_mc, x='Total MC (Days)', y='Provider Name', hue='Provider Name', palette='Greens_d', legend=False, ax=ax)
            plt.title('Top 10 Providers by Total MC (Days)')
            for i, v in enumerate(top_providers_mc['Total MC (Days)']):
                ax.text(v + 0.02 * max(top_providers_mc['Total MC (Days)'], default=0), i, f'{v}', va='center', ha='left', color='black', fontsize=10)

        plt.tight_layout()
        fig = plt.gcf()
        fig.canvas.draw()
        img_array = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
        img_array = img_array.reshape(fig.canvas.get_width_height()[::-1] + (4,))
        plt.close(fig)
        return img_array

    except Exception as e:
        print(f"Plotting error: {str(e)}")
        plt.close()
        return None

# --- Gradio Interface ---
with gr.Blocks(title="Claims Analysis Dashboard") as demo:
    gr.Markdown("# Claims Analysis Dashboard")
    
    with gr.Row():
        url_input = gr.Textbox(label="Website URL")
        user_id_input = gr.Textbox(label="User ID")
        password_input = gr.Textbox(label="Password", type="password")
    with gr.Row():
        start_date_patient = gr.Textbox(label="Start Date (Patient Analysis, YYYY-MM-DD)")
        end_date_patient = gr.Textbox(label="End Date (Patient Analysis, YYYY-MM-DD)")
    with gr.Row():
        start_date_claim = gr.Textbox(label="Start Date (Claim Summary, YYYY-MM-DD)")
        end_date_claim = gr.Textbox(label="End Date (Claim Summary, YYYY-MM-DD)")
    scrape_btn = gr.Button("Scrape Data")
    
    patient_table = gr.HTML(label="Patient Analysis Data")
    claim_table = gr.HTML(label="Claim Summary Data")
    status_output = gr.Textbox(label="Status")
    
    patient_state = gr.State()
    claim_state = gr.State()
    
    chart_dropdown = gr.Dropdown(
        choices=[
            "Top 10 Providers by Avg Claim per Visit",
            "Top 10 Employees by Avg Claim per Visit",
            "Correlation Matrix",
            "Top 10 Employees by Total Visit",
            "Top 10 Employees by Total Claim",
            "Top 10 Employees by Total MC (Days)",
            "Top 10 Providers by Total Visit",
            "Top 10 Providers by Total Claim",
            "Top 10 Providers by Total MC (Days)"
        ],
        label="Select Chart",
        value=None
    )
    plot_btn = gr.Button("Generate Chart")
    plot_output = gr.Image(label="Selected Chart")
    
    def scrape_and_store(*args):
        patient_df, claim_df, status = scrape_data(*args)
        if patient_df is not None and claim_df is not None:
            return (
                patient_df.to_html(), claim_df.to_html(), status,
                patient_df, claim_df
            )
        return None, None, status, None, None
    
    scrape_btn.click(
        fn=scrape_and_store,
        inputs=[url_input, user_id_input, password_input, start_date_patient, end_date_patient, start_date_claim, end_date_claim],
        outputs=[patient_table, claim_table, status_output, patient_state, claim_state]
    )
    
    def plot_chart(chart_type, patient_df, claim_df):
        if patient_df is None or claim_df is None or chart_type is None:
            return None
        return generate_selected_plot(chart_type, patient_df, claim_df)
    
    plot_btn.click(
        fn=plot_chart,
        inputs=[chart_dropdown, patient_state, claim_state],
        outputs=plot_output
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


Scraped claim data: [{'Provider Name': 'Sri Rampai', 'No of Visits': '504', 'Total Claim': '46888.00', 'Total MC (Days)': '0'}, {'Provider Name': 'Wangsa Delima', 'No of Visits': '317', 'Total Claim': '29805.00', 'Total MC (Days)': '0'}, {'Provider Name': 'Wangsa Maju', 'No of Visits': '173', 'Total Claim': '16887.50', 'Total MC (Days)': '0'}, {'Provider Name': 'Setapak Indah Jaya', 'No of Visits': '167', 'Total Claim': '16478.00', 'Total MC (Days)': '0'}, {'Provider Name': 'Danau Kota, Setapak', 'No of Visits': '89', 'Total Claim': '7734.00', 'Total MC (Days)': '0'}]
claim_df columns: ['Provider Name', 'No of Visits', 'Total Claim', 'Total MC (Days)', 'Avg Claim per Visit']
claim_df head:          Provider Name  No of Visits  Total Claim  Total MC (Days)  \
0           Sri Rampai           504      46888.0                0   
1        Wangsa Delima           317      29805.0                0   
2          Wangsa Maju           173      16887.5                0   
3   Setapak Indah Jay

In [3]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from datetime import datetime
import time
import numpy as np
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException

#--- Scraping Functions (Unchanged) ---
def wait_for_element(driver, locator):
    return WebDriverWait(driver, 10).until(EC.element_to_be_clickable(locator))

def select_date_month_day(driver, date_str, date_input_id):
    date_to_select = datetime.strptime(date_str, '%Y-%m-%d')
    date_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, date_input_id)))
    date_input.click()
    month_select = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-datepicker-month')))
    month_option = month_select.find_element(By.XPATH, f"//option[@value='{date_to_select.month - 1}']")
    month_option.click()
    day = date_to_select.day
    day_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//td[@data-handler='selectDay']/a[text()='{day}']")))
    day_element.click()
    time.sleep(5)

def select_date(driver, date_str, date_input_id):
    date_to_select = datetime.strptime(date_str, '%Y-%m-%d')
    date_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, date_input_id)))
    date_input.click()
    time.sleep(2)
    month_select = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-datepicker-month')))
    month_option = month_select.find_element(By.XPATH, f"//option[@value='{date_to_select.month - 1}']")
    time.sleep(2)
    month_option.click()
    year_select = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-datepicker-year')))
    year_option = year_select.find_element(By.XPATH, f"//option[@value='{date_to_select.year}']")
    year_option.click()
    day = date_to_select.day
    day_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//td[@data-handler='selectDay']/a[text()='{day}']")))
    day_element.click()
    time.sleep(5)

def extract_grid_data_clm_summary(driver):
    data = []
    try:
        total_pages_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "sp_1_pjqgridClmSummbyProv")))
        total_pages = int(total_pages_element.text.strip())
    except:
        return data
    for current_page in range(1, total_pages + 1):
        time.sleep(3)
        driver.execute_script("window.scrollTo(0, 0);")
        grid = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "jqgridClmSummbyProv")))
        rows = WebDriverWait(grid, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "tr.jqgrow")))
        for row in rows:
            try:
                provider_name = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_ProvName']").text
                visits = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_NoOfVisit']").text
                claim = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_ClmAmt']").text
                try:
                    total_mc = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_TotalMC']").text
                except:
                    total_mc = '0'
                data.append({'Provider Name': provider_name, 'No of Visits': visits, 'Total Claim': claim, 'Total MC (Days)': total_mc})
            except Exception as e:
                print(f"Error extracting row: {e}")
                continue
        if current_page < total_pages:
            try:
                next_button_div = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn.btn-sm.btn-default span.fa.fa-forward")))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button_div)
                next_button_div.click()
                WebDriverWait(driver, 10).until(EC.staleness_of(rows[0]))
            except:
                break
    return data

def extract_grid_data_patient_analysis(driver):
    all_data = []
    while True:
        driver.execute_script("window.scrollTo(0, 0);")
        grid = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "jqgridCorpMcAnalysis")))
        rows = grid.find_elements(By.CSS_SELECTOR, "tr.jqgrow")
        for row in rows:
            try:
                employee_name = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_MEM_NAME']").text
                employee_no = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_MEM_EMPID']").text
                division = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_MEM_EMPDIVISION']").text
                total_visit = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalVisit']").text
                total_mc = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalMC']").text
                total_claim_own = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalClaim_Own']").text
                total_claim_dep = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalClaim_Dep']").text
                all_data.append({
                    'Employee Name': employee_name, 'Employee No': employee_no, 'Division/Department': division,
                    'Total Visit': total_visit, 'Total MC (Days)': total_mc, 'Total Claim (Own)': total_claim_own,
                    'Total Claim (Dep)': total_claim_dep
                })
            except:
                continue
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            next_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn.btn-sm.btn-default span.fa.fa-forward")))
            parent_div = next_button.find_element(By.XPATH, "./parent::div")
            if "disabled" in parent_div.get_attribute("class"):
                break
            driver.execute_script("arguments[0].scrollIntoView(true);", parent_div)
            parent_div.click()
            WebDriverWait(driver, 10).until(EC.staleness_of(rows[0]))
        except:
            break
    return all_data

def extract_grid_data_mc(driver):
    data = []
    try:
        total_pages_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "sp_1_jqgrid")))
        total_pages = int(total_pages_element.text.strip())
    except:
        total_pages = 1
    for current_page in range(1, total_pages + 1):
        time.sleep(3)
        driver.execute_script("window.scrollTo(0, 0);")
        grid = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "jqgrid")))
        rows = WebDriverWait(grid, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "tr.jqgrow")))
        for row in rows:
            try:
                provider_name = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgrid_STR_ProvName']").text.strip()
                total_mc_given = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgrid_STR_MC_Given_Count']").text.strip()
                total_visits = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgrid_STR_VISITCount']").text.strip()
                data.append({'Provider': provider_name, 'Total MC Given': total_mc_given, 'No. of Visit': total_visits})
            except:
                continue
        if current_page < total_pages:
            try:
                next_button_div = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn.btn-sm.btn-default span.fa.fa-forward")))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button_div)
                next_button_div.click()
                WebDriverWait(driver, 10).until(EC.staleness_of(rows[0]))
            except:
                break
    return data

def scrape_data(url, user_id, password):
    edge_options = Options()
    driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()), options=edge_options)
    start_year = 2024
    current_date = datetime.now().strftime('%Y-%m-%d')
    patient_data_by_year = {}
    claim_data_by_year = {}
    mc_data_by_year = {}
    try:
        driver.get(url)
        image = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//img[@src='/ClaimEXMVR/Servlet_LoadImage?SFC=loadImage&imageName=icorporate.png']")))
        image.click()
        user_id_field = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.NAME, "txtloginid")))
        user_id_field.send_keys(user_id)
        password_field = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "inputpss")))
        password_field.send_keys(password)
        sign_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn.btn-primary[type='submit']")))
        sign_button.click()
        continue_button = wait_for_element(driver, (By.XPATH, "//button[text()='Continue']"))
        continue_button.click()

        for year in range(start_year, 2025 + 1):
            if year == 2024:
                start_date = "2024-01-01"
                end_date = "2024-12-31"
            else:
                start_date = "2025-01-01"
                end_date = current_date
            productivity_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[span[text()='Productivity Reports']]")))
            productivity_link.click()
            patient_analysis_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Patient_Analysis_Report'][span[text()=' Patient Analysis Report ']]")))
            patient_analysis_link.click()
            select_date(driver, start_date, "txtStartDate")
            select_date(driver, end_date, "txtEndDate")
            search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
            search_button.click()
            time.sleep(5)
            dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
            driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
            select = Select(dropdown)
            select.select_by_value("100")
            time.sleep(10)
            patient_data = extract_grid_data_patient_analysis(driver)
            patient_df = pd.DataFrame(patient_data)
            numeric_cols_patient = ['Total Visit', 'Total MC (Days)', 'Total Claim (Own)', 'Total Claim (Dep)']
            for col in numeric_cols_patient:
                patient_df[col] = pd.to_numeric(patient_df[col], errors='coerce')
            patient_df['Total Claim (Combined)'] = patient_df['Total Claim (Own)'] + patient_df['Total Claim (Dep)']
            patient_df['Avg Claim per Visit'] = patient_df['Total Claim (Combined)'] / patient_df['Total Visit']
            patient_df['Avg MC per Visit'] = patient_df['Total MC (Days)'] / patient_df['Total Visit']
            patient_df['Avg Claim per MC'] = patient_df['Total Claim (Combined)'] / patient_df['Total MC (Days)']
            patient_data_by_year[year] = patient_df

        for year in range(start_year, 2025 + 1):
            if year == 2024:
                start_date = "2024-01-01"
                end_date = "2024-12-31"
            else:
                start_date = "2025-01-01"
                end_date = current_date
            productivity_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[span[text()='Productivity Reports']]")))
            productivity_link.click()
            mc_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/MC_HealthCare_By_Provider'][span[text()=' MC by Provider ']]")))
            mc_link.click()
            time.sleep(2)
            select_date(driver, start_date, "txtStartDate")
            select_date(driver, end_date, "txtEndDate")
            search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
            search_button.click()
            time.sleep(10)
            dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
            driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
            select = Select(dropdown)
            select.select_by_value("100")
            time.sleep(10)
            mc_data = extract_grid_data_mc(driver)
            mc_df = pd.DataFrame(mc_data)
            numeric_cols_mc = ['Total MC Given', 'No. of Visit']
            for col in numeric_cols_mc:
                mc_df[col] = pd.to_numeric(mc_df[col], errors='coerce')
            mc_df['% MC Given'] = (mc_df['Total MC Given'] / mc_df['No. of Visit']) * 100
            mc_data_by_year[year] = mc_df

        start_date = "2024-01-01"
        end_date = "2025-01-01"
        reg_claims_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[.//span[contains(text(), 'Registration') and contains(text(), 'Claims')]]")))
        reg_claims_link.click()
        providers_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Claim_Summary_by_Provider_Analysis'][span[text()=' Claim Summary by Providers ']]")))
        providers_link.click()
        time.sleep(5)
        select_date_month_day(driver, start_date, "txtFromDate")
        select_date_month_day(driver, end_date, "txtToDate")
        search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
        driver.execute_script("arguments[0].click();", search_button)
        time.sleep(10)
        dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
        driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
        select = Select(dropdown)
        select.select_by_value("100")
        time.sleep(5)
        claim_data_2024 = extract_grid_data_clm_summary(driver)
        claim_df_2024 = pd.DataFrame(claim_data_2024)
        numeric_cols_claim = ['No of Visits', 'Total Claim', 'Total MC (Days)']
        for col in numeric_cols_claim:
            claim_df_2024[col] = pd.to_numeric(claim_df_2024[col], errors='coerce')
        claim_df_2024['Avg Claim per Visit'] = claim_df_2024['Total Claim'] / claim_df_2024['No of Visits']
        claim_data_by_year[2024] = claim_df_2024

        start_date = "2024-12-31"
        end_date = current_date
        reg_claims_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[.//span[contains(text(), 'Registration') and contains(text(), 'Claims')]]")))
        reg_claims_link.click()
        providers_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Claim_Summary_by_Provider_Analysis'][span[text()=' Claim Summary by Providers ']]")))
        providers_link.click()
        time.sleep(5)
        select_date_month_day(driver, start_date, "txtFromDate")
        select_date_month_day(driver, end_date, "txtToDate")
        search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
        driver.execute_script("arguments[0].click();", search_button)
        time.sleep(10)
        dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
        driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
        select = Select(dropdown)
        select.select_by_value("100")
        time.sleep(5)
        claim_data_recent = extract_grid_data_clm_summary(driver)
        claim_df_recent = pd.DataFrame(claim_data_recent)
        for col in numeric_cols_claim:
            claim_df_recent[col] = pd.to_numeric(claim_df_recent[col], errors='coerce')
        claim_df_recent['Avg Claim per Visit'] = claim_df_recent['Total Claim'] / claim_df_recent['No of Visits']
        claim_data_by_year[2025] = claim_df_recent

        return patient_data_by_year, claim_data_by_year, mc_data_by_year, "Data scraped successfully!"
    except Exception as e:
        return None, None, None, f"Error: {str(e)}"
    finally:
        driver.quit()

# --- Plotting Functions ---
def generate_dashboard_charts(patient_data_by_year, claim_data_by_year, mc_data_by_year, year):
    if not patient_data_by_year or not claim_data_by_year or not mc_data_by_year:
        return None, None
    
    year_int = int(year)
    patient_df = patient_data_by_year.get(year_int, pd.DataFrame())
    claim_df = claim_data_by_year.get(year_int, pd.DataFrame())
    mc_df = mc_data_by_year.get(year_int, pd.DataFrame())
    
    if patient_df.empty or claim_df.empty or mc_df.empty:
        return None, None
    
    sns.set(style="whitegrid", palette="deep")
    provider_charts = []
    employee_charts = []

    # --- Provider Dashboard Charts ---
    # 1. Total Visits by Providers (Horizontal Bar)
    plt.figure(figsize=(10, 6))
    top_prov_visits = mc_df.sort_values('No. of Visit', ascending=False).head(10)
    sns.barplot(data=top_prov_visits, x='No. of Visit', y='Provider', hue='Provider', palette='Blues_d', legend=False)
    plt.title(f'Top 10 Providers by Total Visits ({year})', fontsize=14, weight='bold')
    plt.xlabel('Total Visits', fontsize=12)
    plt.ylabel('Provider', fontsize=12)
    for i, v in enumerate(top_prov_visits['No. of Visit']):
        plt.text(v + 1, i, f'{int(v)}', va='center', fontsize=10)
    plt.tight_layout()
    provider_charts.append(plt.gcf())
    plt.close()

    # 2. Total MC by Providers (Horizontal Bar)
    plt.figure(figsize=(10, 6))
    top_prov_mc = mc_df.sort_values('Total MC Given', ascending=False).head(10)
    sns.barplot(data=top_prov_mc, x='Total MC Given', y='Provider', hue='Provider', palette='Greens_d', legend=False)
    plt.title(f'Top 10 Providers by Total MC Given ({year})', fontsize=14, weight='bold')
    plt.xlabel('Total MC (Days)', fontsize=12)
    plt.ylabel('Provider', fontsize=12)
    for i, v in enumerate(top_prov_mc['Total MC Given']):
        plt.text(v + 1, i, f'{int(v)}', va='center', fontsize=10)
    plt.tight_layout()
    provider_charts.append(plt.gcf())
    plt.close()

    # 3. % MC Given by Providers (Bar)
    plt.figure(figsize=(10, 6))
    top_prov_mc_pct = mc_df.sort_values('% MC Given', ascending=False).head(10)
    sns.barplot(data=top_prov_mc_pct, x='Provider', y='% MC Given', hue='Provider', palette='Purples_d', legend=False)
    plt.title(f'Top 10 Providers by % MC Given ({year})', fontsize=14, weight='bold')
    plt.ylabel('% MC Given', fontsize=12)
    plt.xlabel('Provider', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_prov_mc_pct['% MC Given']):
        plt.text(i, v + 1, f'{v:.1f}%', ha='center', fontsize=10)
    plt.tight_layout()
    provider_charts.append(plt.gcf())
    plt.close()

    # 4. Total Claim by Providers (Horizontal Bar)
    plt.figure(figsize=(10, 6))
    top_prov_claim = claim_df.sort_values('Total Claim', ascending=False).head(10)
    sns.barplot(data=top_prov_claim, x='Total Claim', y='Provider Name', hue='Provider Name', palette='Oranges_d', legend=False)
    plt.title(f'Top 10 Providers by Total Claim ({year})', fontsize=14, weight='bold')
    plt.xlabel('Total Claim ($)', fontsize=12)
    plt.ylabel('Provider', fontsize=12)
    for i, v in enumerate(top_prov_claim['Total Claim']):
        plt.text(v + 1, i, f'{v:.2f}', va='center', fontsize=10)
    plt.tight_layout()
    provider_charts.append(plt.gcf())
    plt.close()

    # 5. Average Claim per Visit by Providers (Bar)
    plt.figure(figsize=(10, 6))
    top_prov_avg_claim = claim_df.sort_values('Avg Claim per Visit', ascending=False).head(10)
    sns.barplot(data=top_prov_avg_claim, x='Provider Name', y='Avg Claim per Visit', hue='Provider Name', palette='Reds_d', legend=False)
    plt.title(f'Top 10 Providers by Avg Claim per Visit ({year})', fontsize=14, weight='bold')
    plt.ylabel('Avg Claim per Visit ($)', fontsize=12)
    plt.xlabel('Provider', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_prov_avg_claim['Avg Claim per Visit']):
        plt.text(i, v + 0.5, f'{v:.2f}', ha='center', fontsize=10)
    plt.tight_layout()
    provider_charts.append(plt.gcf())
    plt.close()

    # 6. Insight: MC vs Claim Correlation (Scatter)
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=claim_df, x='Total MC (Days)', y='Total Claim', size='No of Visits', hue='Provider Name', palette='viridis', legend=False)
    plt.title(f'MC vs Claim by Provider ({year})', fontsize=14, weight='bold')
    plt.xlabel('Total MC (Days)', fontsize=12)
    plt.ylabel('Total Claim ($)', fontsize=12)
    plt.tight_layout()
    provider_charts.append(plt.gcf())
    plt.close()

    # --- Employee Dashboard Charts ---
    # 1. Total Visits by Employees (Horizontal Bar)
    plt.figure(figsize=(10, 6))
    top_emp_visits = patient_df.sort_values('Total Visit', ascending=False).head(10)
    sns.barplot(data=top_emp_visits, x='Total Visit', y='Employee Name', hue='Employee Name', palette='Blues_d', legend=False)
    plt.title(f'Top 10 Employees by Total Visits ({year})', fontsize=14, weight='bold')
    plt.xlabel('Total Visits', fontsize=12)
    plt.ylabel('Employee', fontsize=12)
    for i, v in enumerate(top_emp_visits['Total Visit']):
        plt.text(v + 0.2, i, f'{int(v)}', va='center', fontsize=10)
    plt.tight_layout()
    employee_charts.append(plt.gcf())
    plt.close()

    # 2. Total Claim by Employees (Horizontal Bar)
    plt.figure(figsize=(10, 6))
    top_emp_claim = patient_df.sort_values('Total Claim (Combined)', ascending=False).head(10)
    sns.barplot(data=top_emp_claim, x='Total Claim (Combined)', y='Employee Name', hue='Employee Name', palette='Oranges_d', legend=False)
    plt.title(f'Top 10 Employees by Total Claim ({year})', fontsize=14, weight='bold')
    plt.xlabel('Total Claim ($)', fontsize=12)
    plt.ylabel('Employee', fontsize=12)
    for i, v in enumerate(top_emp_claim['Total Claim (Combined)']):
        plt.text(v + 1, i, f'{v:.2f}', va='center', fontsize=10)
    plt.tight_layout()
    employee_charts.append(plt.gcf())
    plt.close()

    # 3. Average Claim per Visit by Employees (Bar)
    plt.figure(figsize=(10, 6))
    top_emp_avg_claim = patient_df.sort_values('Avg Claim per Visit', ascending=False).head(10)
    sns.barplot(data=top_emp_avg_claim, x='Employee Name', y='Avg Claim per Visit', hue='Employee Name', palette='Reds_d', legend=False)
    plt.title(f'Top 10 Employees by Avg Claim per Visit ({year})', fontsize=14, weight='bold')
    plt.ylabel('Avg Claim per Visit ($)', fontsize=12)
    plt.xlabel('Employee', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_emp_avg_claim['Avg Claim per Visit']):
        plt.text(i, v + 0.5, f'{v:.2f}', ha='center', fontsize=10)
    plt.tight_layout()
    employee_charts.append(plt.gcf())
    plt.close()

    # 4. Total MC by Employees (Horizontal Bar)
    plt.figure(figsize=(10, 6))
    top_emp_mc = patient_df.sort_values('Total MC (Days)', ascending=False).head(10)
    sns.barplot(data=top_emp_mc, x='Total MC (Days)', y='Employee Name', hue='Employee Name', palette='Greens_d', legend=False)
    plt.title(f'Top 10 Employees by Total MC ({year})', fontsize=14, weight='bold')
    plt.xlabel('Total MC (Days)', fontsize=12)
    plt.ylabel('Employee', fontsize=12)
    for i, v in enumerate(top_emp_mc['Total MC (Days)']):
        plt.text(v + 0.2, i, f'{int(v)}', va='center', fontsize=10)
    plt.tight_layout()
    employee_charts.append(plt.gcf())
    plt.close()

    # 5. Average MC per Visit by Employees (Bar)
    plt.figure(figsize=(10, 6))
    top_emp_avg_mc = patient_df.sort_values('Avg MC per Visit', ascending=False).head(10)
    sns.barplot(data=top_emp_avg_mc, x='Employee Name', y='Avg MC per Visit', hue='Employee Name', palette='Purples_d', legend=False)
    plt.title(f'Top 10 Employees by Avg MC per Visit ({year})', fontsize=14, weight='bold')
    plt.ylabel('Avg MC per Visit (Days)', fontsize=12)
    plt.xlabel('Employee', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_emp_avg_mc['Avg MC per Visit']):
        plt.text(i, v + 0.05, f'{v:.2f}', ha='center', fontsize=10)
    plt.tight_layout()
    employee_charts.append(plt.gcf())
    plt.close()

    # 6. Average Claim per MC by Employees (Bar)
    plt.figure(figsize=(10, 6))
    top_emp_avg_claim_mc = patient_df.sort_values('Avg Claim per MC', ascending=False).head(10)
    sns.barplot(data=top_emp_avg_claim_mc, x='Employee Name', y='Avg Claim per MC', hue='Employee Name', palette='YlOrBr', legend=False)
    plt.title(f'Top 10 Employees by Avg Claim per MC ({year})', fontsize=14, weight='bold')
    plt.ylabel('Avg Claim per MC ($)', fontsize=12)
    plt.xlabel('Employee', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_emp_avg_claim_mc['Avg Claim per MC']):
        plt.text(i, v + 0.5, f'{v:.2f}', ha='center', fontsize=10)
    plt.tight_layout()
    employee_charts.append(plt.gcf())
    plt.close()

    # 7. Insight: Division-wise Claim Distribution (Pie)
    plt.figure(figsize=(10, 6))
    division_claims = patient_df.groupby('Division/Department')['Total Claim (Combined)'].sum()
    plt.pie(division_claims, labels=division_claims.index, autopct='%1.1f%%', colors=sns.color_palette('pastel'), startangle=90)
    plt.title(f'Claim Distribution by Division ({year})', fontsize=14, weight='bold')
    plt.tight_layout()
    employee_charts.append(plt.gcf())
    plt.close()

    provider_images = [fig_to_image(fig) for fig in provider_charts]
    employee_images = [fig_to_image(fig) for fig in employee_charts]
    return provider_images, employee_images

def fig_to_image(fig):
    fig.canvas.draw()
    img_array = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
    img_array = img_array.reshape(fig.canvas.get_width_height()[::-1] + (4,))
    plt.close(fig)
    return img_array

# --- Gradio Interface ---
with gr.Blocks(title="Claims Analysis Dashboard") as demo:
    gr.Markdown("# Claims Analysis Dashboard (2024 - Present)")
    
    with gr.Row():
        url_input = gr.Textbox(label="Website URL")
        user_id_input = gr.Textbox(label="User ID")
        password_input = gr.Textbox(label="Password", type="password")
    scrape_btn = gr.Button("Scrape Data")
    
    year_dropdown = gr.Dropdown(
        label="Select Year to View Data",
        choices=["2024", "2025"],
        value="2024",
        allow_custom_value=False
    )
    status_output = gr.Textbox(label="Status")
    
    patient_state = gr.State()
    claim_state = gr.State()
    mc_state = gr.State()

    with gr.Tabs():
        with gr.TabItem("Provider Insights"):
            gr.Markdown(f"## Provider Insights Dashboard")
            with gr.Row():
                prov_chart1 = gr.Image(label="Total Visits by Providers", interactive=False)
                prov_chart2 = gr.Image(label="Total MC by Providers", interactive=False)
            with gr.Row():
                prov_chart3 = gr.Image(label="% MC Given by Providers", interactive=False)
                prov_chart4 = gr.Image(label="Total Claim by Providers", interactive=False)
            with gr.Row():
                prov_chart5 = gr.Image(label="Average Claim per Visit by Providers", interactive=False)
                prov_chart6 = gr.Image(label="MC vs Claim Correlation", interactive=False)

        with gr.TabItem("Employee Insights"):
            gr.Markdown(f"## Employee Insights Dashboard")
            with gr.Row():
                emp_chart1 = gr.Image(label="Total Visits by Employees", interactive=False)
                emp_chart2 = gr.Image(label="Total Claim by Employees", interactive=False)
            with gr.Row():
                emp_chart3 = gr.Image(label="Average Claim per Visit by Employees", interactive=False)
                emp_chart4 = gr.Image(label="Total MC by Employees", interactive=False)
            with gr.Row():
                emp_chart5 = gr.Image(label="Average MC per Visit by Employees", interactive=False)
                emp_chart6 = gr.Image(label="Average Claim per MC by Employees", interactive=False)
            with gr.Row():
                emp_chart7 = gr.Image(label="Claim Distribution by Division", interactive=False)

    def scrape_and_store(url, user_id, password):
        patient_data_by_year, claim_data_by_year, mc_data_by_year, status = scrape_data(url, user_id, password)
        if patient_data_by_year is None or claim_data_by_year is None or mc_data_by_year is None:
            return status, None, None, None
        
        provider_images, employee_images = generate_dashboard_charts(patient_data_by_year, claim_data_by_year, mc_data_by_year, "2024")
        return (
            status, patient_data_by_year, claim_data_by_year, mc_data_by_year,
            provider_images[0], provider_images[1], provider_images[2], provider_images[3], provider_images[4], provider_images[5],
            employee_images[0], employee_images[1], employee_images[2], employee_images[3], employee_images[4], employee_images[5], employee_images[6]
        )

    def update_dashboard(year, patient_data_by_year, claim_data_by_year, mc_data_by_year):
        if not patient_data_by_year or not claim_data_by_year or not mc_data_by_year:
            return [None] * 13
        provider_images, employee_images = generate_dashboard_charts(patient_data_by_year, claim_data_by_year, mc_data_by_year, year)
        return (
            provider_images[0], provider_images[1], provider_images[2], provider_images[3], provider_images[4], provider_images[5],
            employee_images[0], employee_images[1], employee_images[2], employee_images[3], employee_images[4], employee_images[5], employee_images[6]
        )

    scrape_btn.click(
        fn=scrape_and_store,
        inputs=[url_input, user_id_input, password_input],
        outputs=[
            status_output, patient_state, claim_state, mc_state,
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5, prov_chart6,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6, emp_chart7
        ]
    )

    year_dropdown.change(
        fn=update_dashboard,
        inputs=[year_dropdown, patient_state, claim_state, mc_state],
        outputs=[
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5, prov_chart6,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6, emp_chart7
        ]
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [1]:
%%writefile app.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from datetime import datetime
import time
import numpy as np
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException

# --- Scraping Functions ---
def wait_for_element(driver, locator):
    return WebDriverWait(driver, 10).until(EC.element_to_be_clickable(locator))

def select_date_month_day(driver, date_str, date_input_id):
    try:
        date_to_select = datetime.strptime(date_str, '%Y-%m-%d')
        date_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, date_input_id)))
        date_input.click()
        month_select = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-datepicker-month')))
        month_option = month_select.find_element(By.XPATH, f"//option[@value='{date_to_select.month - 1}']")
        month_option.click()
        day = date_to_select.day
        day_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//td[@data-handler='selectDay']/a[text()='{day}']")))
        day_element.click()
        time.sleep(5)
    except Exception as e:
        print(f"Error in select_date_month_day: {str(e)}")
        raise

def select_date(driver, date_str, date_input_id):
    try:
        date_to_select = datetime.strptime(date_str, '%Y-%m-%d')
        date_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, date_input_id)))
        date_input.click()
        time.sleep(2)
        month_select = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-datepicker-month')))
        month_option = month_select.find_element(By.XPATH, f"//option[@value='{date_to_select.month - 1}']")
        time.sleep(2)
        month_option.click()
        year_select = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-datepicker-year')))
        year_option = year_select.find_element(By.XPATH, f"//option[@value='{date_to_select.year}']")
        year_option.click()
        day = date_to_select.day
        day_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//td[@data-handler='selectDay']/a[text()='{day}']")))
        day_element.click()
        time.sleep(5)
    except Exception as e:
        print(f"Error in select_date: {str(e)}")
        raise

def extract_grid_data_clm_summary(driver):
    data = []
    try:
        total_pages_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "sp_1_pjqgridClmSummbyProv")))
        total_pages = int(total_pages_element.text.strip())
    except:
        return data
    for current_page in range(1, total_pages + 1):
        time.sleep(3)
        driver.execute_script("window.scrollTo(0, 0);")
        grid = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "jqgridClmSummbyProv")))
        rows = WebDriverWait(grid, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "tr.jqgrow")))
        for row in rows:
            try:
                provider_name = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_ProvName']").text
                visits = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_NoOfVisit']").text
                claim = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_ClmAmt']").text
                total_mc = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_TotalMC']").text if \
                           row.find_elements(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_TotalMC']") else '0'
                data.append({'Provider Name': provider_name, 'No of Visits': visits, 'Total Claim': claim, 'Total MC (Days)': total_mc})
            except Exception as e:
                print(f"Error extracting row: {e}")
                continue
        if current_page < total_pages:
            try:
                next_button_div = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn.btn-sm.btn-default span.fa.fa-forward")))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button_div)
                next_button_div.click()
                WebDriverWait(driver, 10).until(EC.staleness_of(rows[0]))
            except:
                break
    return data

def extract_grid_data_patient_analysis(driver):
    all_data = []
    while True:
        driver.execute_script("window.scrollTo(0, 0);")
        grid = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "jqgridCorpMcAnalysis")))
        rows = grid.find_elements(By.CSS_SELECTOR, "tr.jqgrow")
        for row in rows:
            try:
                employee_name = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_MEM_NAME']").text
                employee_no = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_MEM_EMPID']").text
                division = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_MEM_EMPDIVISION']").text
                total_visit = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalVisit']").text
                total_mc = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalMC']").text
                total_claim_own = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalClaim_Own']").text
                total_claim_dep = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalClaim_Dep']").text
                all_data.append({
                    'Employee Name': employee_name, 'Employee No': employee_no, 'Division/Department': division,
                    'Total Visit': total_visit, 'Total MC (Days)': total_mc, 'Total Claim (Own)': total_claim_own,
                    'Total Claim (Dep)': total_claim_dep
                })
            except:
                continue
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            next_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn.btn-sm.btn-default span.fa.fa-forward")))
            parent_div = next_button.find_element(By.XPATH, "./parent::div")
            if "disabled" in parent_div.get_attribute("class"):
                break
            driver.execute_script("arguments[0].scrollIntoView(true);", parent_div)
            parent_div.click()
            WebDriverWait(driver, 10).until(EC.staleness_of(rows[0]))
        except:
            break
    return all_data

def extract_grid_data_mc(driver):
    data = []
    try:
        total_pages_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "sp_1_jqgrid")))
        total_pages = int(total_pages_element.text.strip())
    except:
        total_pages = 1
    for current_page in range(1, total_pages + 1):
        time.sleep(3)
        driver.execute_script("window.scrollTo(0, 0);")
        grid = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "jqgrid")))
        rows = WebDriverWait(grid, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "tr.jqgrow")))
        for row in rows:
            try:
                provider_name = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgrid_STR_ProvName']").text.strip()
                total_mc_given = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgrid_STR_MC_Given_Count']").text.strip()
                total_visits = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgrid_STR_VISITCount']").text.strip()
                data.append({'Provider': provider_name, 'Total MC Given': total_mc_given, 'No. of Visit': total_visits})
            except:
                continue
        if current_page < total_pages:
            try:
                next_button_div = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn.btn-sm.btn-default span.fa.fa-forward")))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button_div)
                next_button_div.click()
                WebDriverWait(driver, 10).until(EC.staleness_of(rows[0]))
            except:
                break
    return data

def scrape_data(url, user_id, password):
    edge_options = Options()
    edge_options.add_argument('--headless')
    driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()), options=edge_options)
    start_year = 2024
    current_date = datetime.now().strftime('%Y-%m-%d')
    patient_data_by_year = {}
    claim_data_by_year = {}
    mc_data_by_year = {}
    try:
        driver.get(url)
        image = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//img[@src='/ClaimEXMVR/Servlet_LoadImage?SFC=loadImage&imageName=icorporate.png']")))
        image.click()
        user_id_field = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.NAME, "txtloginid")))
        user_id_field.send_keys(user_id)
        password_field = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "inputpss")))
        password_field.send_keys(password)
        sign_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn.btn-primary[type='submit']")))
        sign_button.click()
        continue_button = wait_for_element(driver, (By.XPATH, "//button[text()='Continue']"))
        continue_button.click()

        for year in range(start_year, 2025 + 1):
            if year == 2024:
                start_date = "2024-01-01"
                end_date = "2024-12-31"
            else:
                start_date = "2025-01-01"
                end_date = current_date
            productivity_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[span[text()='Productivity Reports']]")))
            productivity_link.click()
            patient_analysis_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Patient_Analysis_Report'][span[text()=' Patient Analysis Report ']]")))
            patient_analysis_link.click()
            select_date(driver, start_date, "txtStartDate")
            select_date(driver, end_date, "txtEndDate")
            search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
            search_button.click()
            time.sleep(5)
            dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
            driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
            select = Select(dropdown)
            select.select_by_value("100")
            time.sleep(10)
            patient_data = extract_grid_data_patient_analysis(driver)
            patient_df = pd.DataFrame(patient_data)
            numeric_cols_patient = ['Total Visit', 'Total MC (Days)', 'Total Claim (Own)', 'Total Claim (Dep)']
            for col in numeric_cols_patient:
                patient_df[col] = pd.to_numeric(patient_df[col], errors='coerce')
            patient_df['Total Claim (Combined)'] = patient_df['Total Claim (Own)'] + patient_df['Total Claim (Dep)']
            patient_df['Avg Claim per Visit'] = patient_df['Total Claim (Combined)'] / patient_df['Total Visit']
            patient_df['Avg MC per Visit'] = patient_df['Total MC (Days)'] / patient_df['Total Visit']
            patient_df['Avg Claim per MC'] = patient_df['Total Claim (Combined)'] / patient_df['Total MC (Days)']
            patient_data_by_year[year] = patient_df

        for year in range(start_year, 2025 + 1):
            if year == 2024:
                start_date = "2024-01-01"
                end_date = "2024-12-31"
            else:
                start_date = "2025-01-01"
                end_date = current_date
            productivity_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[span[text()='Productivity Reports']]")))
            productivity_link.click()
            mc_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/MC_HealthCare_By_Provider'][span[text()=' MC by Provider ']]")))
            mc_link.click()
            time.sleep(2)
            select_date(driver, start_date, "txtStartDate")
            select_date(driver, end_date, "txtEndDate")
            search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
            search_button.click()
            time.sleep(10)
            dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
            driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
            select = Select(dropdown)
            select.select_by_value("100")
            time.sleep(10)
            mc_data = extract_grid_data_mc(driver)
            mc_df = pd.DataFrame(mc_data)
            numeric_cols_mc = ['Total MC Given', 'No. of Visit']
            for col in numeric_cols_mc:
                mc_df[col] = pd.to_numeric(mc_df[col], errors='coerce')
            mc_df['% MC Given'] = (mc_df['Total MC Given'] / mc_df['No. of Visit']) * 100
            mc_data_by_year[year] = mc_df

        start_date = "2024-01-01"
        end_date = "2025-01-01"
        reg_claims_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[.//span[contains(text(), 'Registration') and contains(text(), 'Claims')]]")))
        reg_claims_link.click()
        providers_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Claim_Summary_by_Provider_Analysis'][span[text()=' Claim Summary by Providers ']]")))
        providers_link.click()
        time.sleep(5)
        select_date_month_day(driver, start_date, "txtFromDate")
        select_date_month_day(driver, end_date, "txtToDate")
        search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
        driver.execute_script("arguments[0].click();", search_button)
        time.sleep(10)
        dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
        driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
        select = Select(dropdown)
        select.select_by_value("100")
        time.sleep(5)
        claim_data_2024 = extract_grid_data_clm_summary(driver)
        claim_df_2024 = pd.DataFrame(claim_data_2024)
        numeric_cols_claim = ['No of Visits', 'Total Claim', 'Total MC (Days)']
        for col in numeric_cols_claim:
            claim_df_2024[col] = pd.to_numeric(claim_df_2024[col], errors='coerce')
        claim_df_2024['Avg Claim per Visit'] = claim_df_2024['Total Claim'] / claim_df_2024['No of Visits']
        claim_data_by_year[2024] = claim_df_2024

        start_date = "2024-12-31"
        end_date = current_date
        reg_claims_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[.//span[contains(text(), 'Registration') and contains(text(), 'Claims')]]")))
        reg_claims_link.click()
        providers_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Claim_Summary_by_Provider_Analysis'][span[text()=' Claim Summary by Providers ']]")))
        providers_link.click()
        time.sleep(5)
        select_date_month_day(driver, start_date, "txtFromDate")
        select_date_month_day(driver, end_date, "txtToDate")
        search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
        driver.execute_script("arguments[0].click();", search_button)
        time.sleep(10)
        dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
        driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
        select = Select(dropdown)
        select.select_by_value("100")
        time.sleep(5)
        claim_data_recent = extract_grid_data_clm_summary(driver)
        claim_df_recent = pd.DataFrame(claim_data_recent)
        for col in numeric_cols_claim:
            claim_df_recent[col] = pd.to_numeric(claim_df_recent[col], errors='coerce')
        claim_df_recent['Avg Claim per Visit'] = claim_df_recent['Total Claim'] / claim_df_recent['No of Visits']
        claim_data_by_year[2025] = claim_df_recent

        return patient_data_by_year, claim_data_by_year, mc_data_by_year, "Data scraped successfully!"
    except Exception as e:
        return None, None, None, f"Error: {str(e)}"
    finally:
        driver.quit()

# --- Plotting Functions ---
def generate_dashboard_charts(patient_data_by_year, claim_data_by_year, mc_data_by_year, year):
    if not patient_data_by_year or not claim_data_by_year or not mc_data_by_year:
        return None, None
    
    year_int = int(year)
    patient_df = patient_data_by_year.get(year_int, pd.DataFrame())
    claim_df = claim_data_by_year.get(year_int, pd.DataFrame())
    mc_df = mc_data_by_year.get(year_int, pd.DataFrame())
    
    if patient_df.empty or claim_df.empty or mc_df.empty:
        return None, None
    
    sns.set(style="whitegrid", palette="deep")
    provider_charts = []
    employee_charts = []

    # --- Provider Dashboard Charts ---
    # 1. Total Visits by Providers
    fig, ax = plt.subplots(figsize=(10, 6))
    top_prov_visits = mc_df.sort_values('No. of Visit', ascending=False).head(10)
    sns.barplot(data=top_prov_visits, x='No. of Visit', y='Provider', hue='Provider', palette='Blues_d', legend=False)
    plt.title(f'Top 10 Providers by Total Visits ({year})', fontsize=14, weight='bold')
    plt.xlabel('Total Visits', fontsize=12)
    plt.ylabel('Provider', fontsize=12)
    for i, v in enumerate(top_prov_visits['No. of Visit']):
        plt.text(v + 1, i, f'{int(v)}', va='center', fontsize=10)
    plt.tight_layout()
    provider_charts.append(fig)

    # 2. Total MC by Providers
    fig, ax = plt.subplots(figsize=(10, 6))
    top_prov_mc = mc_df.sort_values('Total MC Given', ascending=False).head(10)
    sns.barplot(data=top_prov_mc, x='Total MC Given', y='Provider', hue='Provider', palette='Greens_d', legend=False)
    plt.title(f'Top 10 Providers by Total MC Given ({year})', fontsize=14, weight='bold')
    plt.xlabel('Total MC (Days)', fontsize=12)
    plt.ylabel('Provider', fontsize=12)
    for i, v in enumerate(top_prov_mc['Total MC Given']):
        plt.text(v + 1, i, f'{int(v)}', va='center', fontsize=10)
    plt.tight_layout()
    provider_charts.append(fig)

    # 3. % MC Given by Providers
    fig, ax = plt.subplots(figsize=(10, 6))
    top_prov_mc_pct = mc_df.sort_values('% MC Given', ascending=False).head(10)
    sns.barplot(data=top_prov_mc_pct, x='Provider', y='% MC Given', hue='Provider', palette='Purples_d', legend=False)
    plt.title(f'Top 10 Providers by % MC Given ({year})', fontsize=14, weight='bold')
    plt.ylabel('% MC Given', fontsize=12)
    plt.xlabel('Provider', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_prov_mc_pct['% MC Given']):
        plt.text(i, v + 1, f'{v:.1f}%', ha='center', fontsize=10)
    plt.tight_layout()
    provider_charts.append(fig)

    # 4. Total Claim by Providers
    fig, ax = plt.subplots(figsize=(10, 6))
    top_prov_claim = claim_df.sort_values('Total Claim', ascending=False).head(10)
    sns.barplot(data=top_prov_claim, x='Total Claim', y='Provider Name', hue='Provider Name', palette='Oranges_d', legend=False)
    plt.title(f'Top 10 Providers by Total Claim ({year})', fontsize=14, weight='bold')
    plt.xlabel('Total Claim ($)', fontsize=12)
    plt.ylabel('Provider', fontsize=12)
    for i, v in enumerate(top_prov_claim['Total Claim']):
        plt.text(v + 1, i, f'{v:.2f}', va='center', fontsize=10)
    plt.tight_layout()
    provider_charts.append(fig)

    # 5. Average Claim per Visit by Providers
    fig, ax = plt.subplots(figsize=(10, 6))
    top_prov_avg_claim = claim_df.sort_values('Avg Claim per Visit', ascending=False).head(10)
    sns.barplot(data=top_prov_avg_claim, x='Provider Name', y='Avg Claim per Visit', hue='Provider Name', palette='Reds_d', legend=False)
    plt.title(f'Top 10 Providers by Avg Claim per Visit ({year})', fontsize=14, weight='bold')
    plt.ylabel('Avg Claim per Visit ($)', fontsize=12)
    plt.xlabel('Provider', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_prov_avg_claim['Avg Claim per Visit']):
        plt.text(i, v + 0.5, f'{v:.2f}', ha='center', fontsize=10)
    plt.tight_layout()
    provider_charts.append(fig)

    # 6. MC vs Claim Correlation
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.scatterplot(data=claim_df, x='Total MC (Days)', y='Total Claim', size='No of Visits', hue='Provider Name', palette='viridis', legend=False)
    plt.title(f'MC vs Claim by Provider ({year})', fontsize=14, weight='bold')
    plt.xlabel('Total MC (Days)', fontsize=12)
    plt.ylabel('Total Claim ($)', fontsize=12)
    plt.tight_layout()
    provider_charts.append(fig)

    # --- Employee Dashboard Charts ---
    # 1. Total Visits by Employees
    fig, ax = plt.subplots(figsize=(10, 6))
    top_emp_visits = patient_df.sort_values('Total Visit', ascending=False).head(10)
    sns.barplot(data=top_emp_visits, x='Total Visit', y='Employee Name', hue='Employee Name', palette='Blues_d', legend=False)
    plt.title(f'Top 10 Employees by Total Visits ({year})', fontsize=14, weight='bold')
    plt.xlabel('Total Visits', fontsize=12)
    plt.ylabel('Employee', fontsize=12)
    for i, v in enumerate(top_emp_visits['Total Visit']):
        plt.text(v + 0.2, i, f'{int(v)}', va='center', fontsize=10)
    plt.tight_layout()
    employee_charts.append(fig)

    # 2. Total Claim by Employees
    fig, ax = plt.subplots(figsize=(10, 6))
    top_emp_claim = patient_df.sort_values('Total Claim (Combined)', ascending=False).head(10)
    sns.barplot(data=top_emp_claim, x='Total Claim (Combined)', y='Employee Name', hue='Employee Name', palette='Oranges_d', legend=False)
    plt.title(f'Top 10 Employees by Total Claim ({year})', fontsize=14, weight='bold')
    plt.xlabel('Total Claim ($)', fontsize=12)
    plt.ylabel('Employee', fontsize=12)
    for i, v in enumerate(top_emp_claim['Total Claim (Combined)']):
        plt.text(v + 1, i, f'{v:.2f}', va='center', fontsize=10)
    plt.tight_layout()
    employee_charts.append(fig)

    # 3. Average Claim per Visit by Employees
    fig, ax = plt.subplots(figsize=(10, 6))
    top_emp_avg_claim = patient_df.sort_values('Avg Claim per Visit', ascending=False).head(10)
    sns.barplot(data=top_emp_avg_claim, x='Employee Name', y='Avg Claim per Visit', hue='Employee Name', palette='Reds_d', legend=False)
    plt.title(f'Top 10 Employees by Avg Claim per Visit ({year})', fontsize=14, weight='bold')
    plt.ylabel('Avg Claim per Visit ($)', fontsize=12)
    plt.xlabel('Employee', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_emp_avg_claim['Avg Claim per Visit']):
        plt.text(i, v + 0.5, f'{v:.2f}', ha='center', fontsize=10)
    plt.tight_layout()
    employee_charts.append(fig)

    # 4. Total MC by Employees
    fig, ax = plt.subplots(figsize=(10, 6))
    top_emp_mc = patient_df.sort_values('Total MC (Days)', ascending=False).head(10)
    sns.barplot(data=top_emp_mc, x='Total MC (Days)', y='Employee Name', hue='Employee Name', palette='Greens_d', legend=False)
    plt.title(f'Top 10 Employees by Total MC ({year})', fontsize=14, weight='bold')
    plt.xlabel('Total MC (Days)', fontsize=12)
    plt.ylabel('Employee', fontsize=12)
    for i, v in enumerate(top_emp_mc['Total MC (Days)']):
        plt.text(v + 0.2, i, f'{int(v)}', va='center', fontsize=10)
    plt.tight_layout()
    employee_charts.append(fig)

    # 5. Average MC per Visit by Employees
    fig, ax = plt.subplots(figsize=(10, 6))
    top_emp_avg_mc = patient_df.sort_values('Avg MC per Visit', ascending=False).head(10)
    sns.barplot(data=top_emp_avg_mc, x='Employee Name', y='Avg MC per Visit', hue='Employee Name', palette='Purples_d', legend=False)
    plt.title(f'Top 10 Employees by Avg MC per Visit ({year})', fontsize=14, weight='bold')
    plt.ylabel('Avg MC per Visit (Days)', fontsize=12)
    plt.xlabel('Employee', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_emp_avg_mc['Avg MC per Visit']):
        plt.text(i, v + 0.05, f'{v:.2f}', ha='center', fontsize=10)
    plt.tight_layout()
    employee_charts.append(fig)

    # 6. Average Claim per MC by Employees
    fig, ax = plt.subplots(figsize=(10, 6))
    top_emp_avg_claim_mc = patient_df.sort_values('Avg Claim per MC', ascending=False).head(10)
    sns.barplot(data=top_emp_avg_claim_mc, x='Employee Name', y='Avg Claim per MC', hue='Employee Name', palette='YlOrBr', legend=False)
    plt.title(f'Top 10 Employees by Avg Claim per MC ({year})', fontsize=14, weight='bold')
    plt.ylabel('Avg Claim per MC ($)', fontsize=12)
    plt.xlabel('Employee', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_emp_avg_claim_mc['Avg Claim per MC']):
        plt.text(i, v + 0.5, f'{v:.2f}', ha='center', fontsize=10)
    plt.tight_layout()
    employee_charts.append(fig)

    # 7. Division-wise Claim Distribution
    fig, ax = plt.subplots(figsize=(10, 6))
    division_claims = patient_df.groupby('Division/Department')['Total Claim (Combined)'].sum()
    plt.pie(division_claims, labels=division_claims.index, autopct='%1.1f%%', colors=sns.color_palette('pastel'), startangle=90)
    plt.title(f'Claim Distribution by Division ({year})', fontsize=14, weight='bold')
    plt.tight_layout()
    employee_charts.append(fig)

    return provider_charts, employee_charts

# --- Streamlit Interface ---
def main():
    st.set_page_config(page_title="Claims Analysis Dashboard", layout="wide")

    if 'data_loaded' not in st.session_state:
        st.session_state['data_loaded'] = False
        st.session_state['patient_data'] = {}
        st.session_state['claim_data'] = {}
        st.session_state['mc_data'] = {}
        st.session_state['selected_year'] = "2024"

    with st.sidebar:
        st.title("Claims Analysis Dashboard")
        st.subheader("Login Details")
        
        url = st.text_input("Website URL", key="url")
        user_id = st.text_input("User ID", key="user_id")
        password = st.text_input("Password", type="password", key="password")
        
        if st.button("Scrape Data", key="scrape_button"):
            if not all([url, user_id, password]):
                st.error("Please fill in all login details")
            else:
                with st.spinner("Scraping data... This may take a few minutes."):
                    patient_data_by_year, claim_data_by_year, mc_data_by_year, status = scrape_data(url, user_id, password)
                    if patient_data_by_year and claim_data_by_year and mc_data_by_year:
                        st.session_state['patient_data'] = patient_data_by_year
                        st.session_state['claim_data'] = claim_data_by_year
                        st.session_state['mc_data'] = mc_data_by_year
                        st.session_state['data_loaded'] = True
                        st.success(status)
                    else:
                        st.error(status)
                        st.session_state['data_loaded'] = False
        
        if st.session_state['data_loaded']:
            year = st.selectbox("Select Year", options=["2024", "2025"], index=0, key="year_select")
            st.session_state['selected_year'] = year

    st.title("Healthcare Claims Analysis Dashboard")
    
    if not st.session_state['data_loaded']:
        st.info("Please enter your credentials and click 'Scrape Data' to begin.")
        return

    tab1, tab2 = st.tabs(["Provider Insights", "Employee Insights"])
    year = st.session_state['selected_year']
    provider_charts, employee_charts = generate_dashboard_charts(
        st.session_state['patient_data'], st.session_state['claim_data'], st.session_state['mc_data'], year
    )

    if not provider_charts or not employee_charts:
        st.error("Failed to generate charts. Please check the scraped data.")
        return

    with tab1:
        st.header(f"Provider Insights Dashboard ({year})")
        col1, col2 = st.columns(2)
        for i in range(0, len(provider_charts), 2):
            with col1:
                st.pyplot(provider_charts[i])
            if i + 1 < len(provider_charts):
                with col2:
                    st.pyplot(provider_charts[i + 1])

    with tab2:
        st.header(f"Employee Insights Dashboard ({year})")
        col1, col2 = st.columns(2)
        for i in range(0, len(employee_charts), 2):
            with col1:
                st.pyplot(employee_charts[i])
            if i + 1 < len(employee_charts):
                with col2:
                    st.pyplot(employee_charts[i + 1])

if __name__ == "__main__":
    main()

Overwriting app.py


In [ ]:
!streamlit run app.py --server.headless true

In [5]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from datetime import datetime
import time
import numpy as np
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException

#--- Scraping Functions (Unchanged) ---
def wait_for_element(driver, locator):
    return WebDriverWait(driver, 10).until(EC.element_to_be_clickable(locator))

def select_date_month_day(driver, date_str, date_input_id):
    date_to_select = datetime.strptime(date_str, '%Y-%m-%d')
    date_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, date_input_id)))
    date_input.click()
    month_select = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-datepicker-month')))
    month_option = month_select.find_element(By.XPATH, f"//option[@value='{date_to_select.month - 1}']")
    month_option.click()
    day = date_to_select.day
    day_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//td[@data-handler='selectDay']/a[text()='{day}']")))
    day_element.click()
    time.sleep(5)

def select_date(driver, date_str, date_input_id):
    date_to_select = datetime.strptime(date_str, '%Y-%m-%d')
    date_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, date_input_id)))
    date_input.click()
    time.sleep(2)
    month_select = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-datepicker-month')))
    month_option = month_select.find_element(By.XPATH, f"//option[@value='{date_to_select.month - 1}']")
    time.sleep(2)
    month_option.click()
    year_select = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-datepicker-year')))
    year_option = year_select.find_element(By.XPATH, f"//option[@value='{date_to_select.year}']")
    year_option.click()
    day = date_to_select.day
    day_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//td[@data-handler='selectDay']/a[text()='{day}']")))
    day_element.click()
    time.sleep(5)

def extract_grid_data_clm_summary(driver):
    data = []
    try:
        total_pages_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "sp_1_pjqgridClmSummbyProv")))
        total_pages = int(total_pages_element.text.strip())
    except:
        return data
    for current_page in range(1, total_pages + 1):
        time.sleep(3)
        driver.execute_script("window.scrollTo(0, 0);")
        grid = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "jqgridClmSummbyProv")))
        rows = WebDriverWait(grid, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "tr.jqgrow")))
        for row in rows:
            try:
                provider_name = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_ProvName']").text
                visits = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_NoOfVisit']").text
                claim = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_ClmAmt']").text
                try:
                    total_mc = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_TotalMC']").text
                except:
                    total_mc = '0'
                data.append({'Provider Name': provider_name, 'No of Visits': visits, 'Total Claim': claim, 'Total MC (Days)': total_mc})
            except Exception as e:
                print(f"Error extracting row: {e}")
                continue
        if current_page < total_pages:
            try:
                next_button_div = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn.btn-sm.btn-default span.fa.fa-forward")))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button_div)
                next_button_div.click()
                WebDriverWait(driver, 10).until(EC.staleness_of(rows[0]))
            except:
                break
    return data

def extract_grid_data_patient_analysis(driver):
    all_data = []
    while True:
        driver.execute_script("window.scrollTo(0, 0);")
        grid = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "jqgridCorpMcAnalysis")))
        rows = grid.find_elements(By.CSS_SELECTOR, "tr.jqgrow")
        for row in rows:
            try:
                employee_name = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_MEM_NAME']").text
                employee_no = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_MEM_EMPID']").text
                division = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_MEM_EMPDIVISION']").text
                total_visit = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalVisit']").text
                total_mc = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalMC']").text
                total_claim_own = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalClaim_Own']").text
                total_claim_dep = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalClaim_Dep']").text
                all_data.append({
                    'Employee Name': employee_name, 'Employee No': employee_no, 'Division/Department': division,
                    'Total Visit': total_visit, 'Total MC (Days)': total_mc, 'Total Claim (Own)': total_claim_own,
                    'Total Claim (Dep)': total_claim_dep
                })
            except:
                continue
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            next_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn.btn-sm.btn-default span.fa.fa-forward")))
            parent_div = next_button.find_element(By.XPATH, "./parent::div")
            if "disabled" in parent_div.get_attribute("class"):
                break
            driver.execute_script("arguments[0].scrollIntoView(true);", parent_div)
            parent_div.click()
            WebDriverWait(driver, 10).until(EC.staleness_of(rows[0]))
        except:
            break
    return all_data

def extract_grid_data_mc(driver):
    data = []
    try:
        total_pages_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "sp_1_jqgrid")))
        total_pages = int(total_pages_element.text.strip())
    except:
        total_pages = 1
    for current_page in range(1, total_pages + 1):
        time.sleep(3)
        driver.execute_script("window.scrollTo(0, 0);")
        grid = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "jqgrid")))
        rows = WebDriverWait(grid, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "tr.jqgrow")))
        for row in rows:
            try:
                provider_name = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgrid_STR_ProvName']").text.strip()
                total_mc_given = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgrid_STR_MC_Given_Count']").text.strip()
                total_visits = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgrid_STR_VISITCount']").text.strip()
                data.append({'Provider': provider_name, 'Total MC Given': total_mc_given, 'No. of Visit': total_visits})
            except:
                continue
        if current_page < total_pages:
            try:
                next_button_div = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn.btn-sm.btn-default span.fa.fa-forward")))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button_div)
                next_button_div.click()
                WebDriverWait(driver, 10).until(EC.staleness_of(rows[0]))
            except:
                break
    return data

def scrape_data(url, user_id, password):
    edge_options = Options()
    driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()), options=edge_options)
    start_year = 2024
    current_date = datetime.now().strftime('%Y-%m-%d')
    patient_data_by_year = {}
    claim_data_by_year = {}
    mc_data_by_year = {}
    try:
        driver.get(url)
        image = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//img[@src='/ClaimEXMVR/Servlet_LoadImage?SFC=loadImage&imageName=icorporate.png']")))
        image.click()
        user_id_field = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.NAME, "txtloginid")))
        user_id_field.send_keys(user_id)
        password_field = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "inputpss")))
        password_field.send_keys(password)
        sign_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn.btn-primary[type='submit']")))
        sign_button.click()
        continue_button = wait_for_element(driver, (By.XPATH, "//button[text()='Continue']"))
        continue_button.click()

        for year in range(start_year, 2025 + 1):
            if year == 2024:
                start_date = "2024-01-01"
                end_date = "2024-12-31"
            else:
                start_date = "2025-01-01"
                end_date = current_date
            productivity_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[span[text()='Productivity Reports']]")))
            productivity_link.click()
            patient_analysis_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Patient_Analysis_Report'][span[text()=' Patient Analysis Report ']]")))
            patient_analysis_link.click()
            select_date(driver, start_date, "txtStartDate")
            select_date(driver, end_date, "txtEndDate")
            search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
            search_button.click()
            time.sleep(5)
            dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
            driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
            select = Select(dropdown)
            select.select_by_value("100")
            time.sleep(10)
            patient_data = extract_grid_data_patient_analysis(driver)
            patient_df = pd.DataFrame(patient_data)
            numeric_cols_patient = ['Total Visit', 'Total MC (Days)', 'Total Claim (Own)', 'Total Claim (Dep)']
            for col in numeric_cols_patient:
                patient_df[col] = pd.to_numeric(patient_df[col], errors='coerce')
            patient_df['Total Claim (Combined)'] = patient_df['Total Claim (Own)'] + patient_df['Total Claim (Dep)']
            patient_df['Avg Claim per Visit'] = patient_df['Total Claim (Combined)'] / patient_df['Total Visit']
            patient_df['Avg MC per Visit'] = patient_df['Total MC (Days)'] / patient_df['Total Visit']
            patient_df['Avg Claim per MC'] = patient_df['Total Claim (Combined)'] / patient_df['Total MC (Days)']
            patient_data_by_year[year] = patient_df

        for year in range(start_year, 2025 + 1):
            if year == 2024:
                start_date = "2024-01-01"
                end_date = "2024-12-31"
            else:
                start_date = "2025-01-01"
                end_date = current_date
            productivity_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[span[text()='Productivity Reports']]")))
            productivity_link.click()
            mc_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/MC_HealthCare_By_Provider'][span[text()=' MC by Provider ']]")))
            mc_link.click()
            time.sleep(2)
            select_date(driver, start_date, "txtStartDate")
            select_date(driver, end_date, "txtEndDate")
            search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
            search_button.click()
            time.sleep(10)
            dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
            driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
            select = Select(dropdown)
            select.select_by_value("100")
            time.sleep(10)
            mc_data = extract_grid_data_mc(driver)
            mc_df = pd.DataFrame(mc_data)
            numeric_cols_mc = ['Total MC Given', 'No. of Visit']
            for col in numeric_cols_mc:
                mc_df[col] = pd.to_numeric(mc_df[col], errors='coerce')
            mc_df['% MC Given'] = (mc_df['Total MC Given'] / mc_df['No. of Visit']) * 100
            mc_data_by_year[year] = mc_df

        start_date = "2024-01-01"
        end_date = "2025-01-01"
        reg_claims_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[.//span[contains(text(), 'Registration') and contains(text(), 'Claims')]]")))
        reg_claims_link.click()
        providers_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Claim_Summary_by_Provider_Analysis'][span[text()=' Claim Summary by Providers ']]")))
        providers_link.click()
        time.sleep(5)
        select_date_month_day(driver, start_date, "txtFromDate")
        select_date_month_day(driver, end_date, "txtToDate")
        search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
        driver.execute_script("arguments[0].click();", search_button)
        time.sleep(10)
        dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
        driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
        select = Select(dropdown)
        select.select_by_value("100")
        time.sleep(5)
        claim_data_2024 = extract_grid_data_clm_summary(driver)
        claim_df_2024 = pd.DataFrame(claim_data_2024)
        numeric_cols_claim = ['No of Visits', 'Total Claim', 'Total MC (Days)']
        for col in numeric_cols_claim:
            claim_df_2024[col] = pd.to_numeric(claim_df_2024[col], errors='coerce')
        claim_df_2024['Avg Claim per Visit'] = claim_df_2024['Total Claim'] / claim_df_2024['No of Visits']
        claim_data_by_year[2024] = claim_df_2024

        start_date = "2024-12-31"
        end_date = current_date
        reg_claims_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[.//span[contains(text(), 'Registration') and contains(text(), 'Claims')]]")))
        reg_claims_link.click()
        providers_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Claim_Summary_by_Provider_Analysis'][span[text()=' Claim Summary by Providers ']]")))
        providers_link.click()
        time.sleep(5)
        select_date_month_day(driver, start_date, "txtFromDate")
        select_date_month_day(driver, end_date, "txtToDate")
        search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
        driver.execute_script("arguments[0].click();", search_button)
        time.sleep(10)
        dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
        driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
        select = Select(dropdown)
        select.select_by_value("100")
        time.sleep(5)
        claim_data_recent = extract_grid_data_clm_summary(driver)
        claim_df_recent = pd.DataFrame(claim_data_recent)
        for col in numeric_cols_claim:
            claim_df_recent[col] = pd.to_numeric(claim_df_recent[col], errors='coerce')
        claim_df_recent['Avg Claim per Visit'] = claim_df_recent['Total Claim'] / claim_df_recent['No of Visits']
        claim_data_by_year[2025] = claim_df_recent

        return patient_data_by_year, claim_data_by_year, mc_data_by_year, "Data scraped successfully!"
    except Exception as e:
        return None, None, None, f"Error: {str(e)}"
    finally:
        driver.quit()


# --- Plotting Functions ---
def generate_dashboard_charts(patient_data_by_year, claim_data_by_year, mc_data_by_year, year, show_mc_pct=True, mc_sort_order="desc"):
    if not patient_data_by_year or not claim_data_by_year or not mc_data_by_year:
        return None, None
    
    year_int = int(year)
    patient_df = patient_data_by_year.get(year_int, pd.DataFrame())
    claim_df = claim_data_by_year.get(year_int, pd.DataFrame())
    mc_df = mc_data_by_year.get(year_int, pd.DataFrame())
    
    if patient_df.empty or claim_df.empty or mc_df.empty:
        return None, None
    
    # Professional styling
    sns.set(style="whitegrid", palette="muted")
    plt.rcParams.update({
        'font.family': 'Helvetica', 'font.size': 12, 'axes.titlesize': 16, 
        'axes.labelsize': 14, 'xtick.labelsize': 11, 'ytick.labelsize': 11,
        'axes.titleweight': 'bold', 'axes.linewidth': 1.5, 'grid.linestyle': ':', 
        'grid.alpha': 0.5, 'figure.facecolor': '#f5f6f5', 'axes.facecolor': '#ffffff',
        'axes.edgecolor': '#333333', 'axes.labelcolor': '#333333', 'text.color': '#333333'
    })
    
    provider_charts = []
    employee_charts = []

    # --- Provider Dashboard Charts ---
    # 1. Total Visits by Providers (Horizontal Bar)
    plt.figure(figsize=(12, 6))
    top_prov_visits = mc_df.sort_values('No. of Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_visits, x='No. of Visit', y='Provider', hue='Provider', palette='Blues_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Total Visits ({year})', pad=15)
    ax.set_xlabel('Total Visits')
    ax.set_ylabel('Provider')
    for i, v in enumerate(top_prov_visits['No. of Visit']):
        ax.text(v + 0.5, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(plt.gcf())
    plt.close()

    # 2. Total MC by Providers (Horizontal Bar)
    plt.figure(figsize=(12, 6))
    top_prov_mc = mc_df.sort_values('Total MC Given', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_mc, x='Total MC Given', y='Provider', hue='Provider', palette='Greens_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Total MC Given ({year})', pad=15)
    ax.set_xlabel('Total MC (Days)')
    ax.set_ylabel('Provider')
    for i, v in enumerate(top_prov_mc['Total MC Given']):
        ax.text(v + 0.5, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(plt.gcf())
    plt.close()

    # 3. % MC Given by Providers (Larger Bar, Top 20, Sortable)
    if show_mc_pct:
        plt.figure(figsize=(18, 9))  # Larger size for professionalism and readability
        top_visits_provs = set(mc_df.sort_values('No. of Visit', ascending=False).head(10)['Provider'])
        top_mc_provs = set(mc_df.sort_values('Total MC Given', ascending=False).head(10)['Provider'])
        top_provs = top_visits_provs.union(top_mc_provs)
        top_prov_mc_pct = mc_df[mc_df['Provider'].isin(top_provs)].sort_values(
            '% MC Given', ascending=(mc_sort_order == "asc")).head(20)
        ax = sns.barplot(data=top_prov_mc_pct, x='Provider', y='% MC Given', hue='Provider', 
                         palette='Purples_r', legend=False, edgecolor='black', linewidth=0.5)
        ax.set_title(f'Top 20 Providers by % MC Given ({year}) - Sorted {"Ascending" if mc_sort_order == "asc" else "Descending"}', pad=15)
        ax.set_ylabel('% MC Given', fontsize=14)
        ax.set_xlabel('Provider', fontsize=14)
        plt.xticks(rotation=45, ha='right', fontsize=11)
        for i, v in enumerate(top_prov_mc_pct['% MC Given']):
            ax.text(i, v + 1, f'{v:.1f}%', ha='center', fontsize=10, color='#333333')
        plt.tight_layout()
        provider_charts.append(plt.gcf())
    else:
        provider_charts.append(None)
    plt.close()

    # 4. Total Claim by Providers (Horizontal Bar)
    plt.figure(figsize=(12, 6))
    top_prov_claim = claim_df.sort_values('Total Claim', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_claim, x='Total Claim', y='Provider Name', hue='Provider Name', 
                     palette='Oranges_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Total Claim ({year})', pad=15)
    ax.set_xlabel('Total Claim ($)')
    ax.set_ylabel('Provider')
    for i, v in enumerate(top_prov_claim['Total Claim']):
        ax.text(v + 0.5, i, f'{v:,.2f}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(plt.gcf())
    plt.close()

    # 5. Average Claim per Visit by Providers (Bar)
    plt.figure(figsize=(12, 6))
    top_prov_avg_claim = claim_df.sort_values('Avg Claim per Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_avg_claim, x='Provider Name', y='Avg Claim per Visit', hue='Provider Name', 
                     palette='Reds_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Avg Claim per Visit ({year})', pad=15)
    ax.set_ylabel('Avg Claim per Visit ($)')
    ax.set_xlabel('Provider')
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_prov_avg_claim['Avg Claim per Visit']):
        ax.text(i, v + 0.5, f'{v:.2f}', ha='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(plt.gcf())
    plt.close()

    # --- Employee Dashboard Charts ---
    # 1. Total Visits by Employees
    plt.figure(figsize=(12, 6))
    top_emp_visits = patient_df.sort_values('Total Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_visits, x='Total Visit', y='Employee Name', hue='Employee Name', 
                     palette='Blues_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Total Visits ({year})', pad=15)
    ax.set_xlabel('Total Visits')
    ax.set_ylabel('Employee')
    for i, v in enumerate(top_emp_visits['Total Visit']):
        ax.text(v + 0.2, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(plt.gcf())
    plt.close()

    # 2. Total Claim by Employees
    plt.figure(figsize=(12, 6))
    top_emp_claim = patient_df.sort_values('Total Claim (Combined)', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_claim, x='Total Claim (Combined)', y='Employee Name', hue='Employee Name', 
                     palette='Oranges_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Total Claim ({year})', pad=15)
    ax.set_xlabel('Total Claim ($)')
    ax.set_ylabel('Employee')
    for i, v in enumerate(top_emp_claim['Total Claim (Combined)']):
        ax.text(v + 1, i, f'{v:,.2f}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(plt.gcf())
    plt.close()

    # 3. Average Claim per Visit by Employees
    plt.figure(figsize=(12, 6))
    top_emp_avg_claim = patient_df.sort_values('Avg Claim per Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_avg_claim, x='Employee Name', y='Avg Claim per Visit', hue='Employee Name', 
                     palette='Reds_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Avg Claim per Visit ({year})', pad=15)
    ax.set_ylabel('Avg Claim per Visit ($)')
    ax.set_xlabel('Employee')
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_emp_avg_claim['Avg Claim per Visit']):
        ax.text(i, v + 0.5, f'{v:.2f}', ha='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(plt.gcf())
    plt.close()

    # 4. Total MC by Employees
    plt.figure(figsize=(12, 6))
    top_emp_mc = patient_df.sort_values('Total MC (Days)', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_mc, x='Total MC (Days)', y='Employee Name', hue='Employee Name', 
                     palette='Greens_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Total MC ({year})', pad=15)
    ax.set_xlabel('Total MC (Days)')
    ax.set_ylabel('Employee')
    for i, v in enumerate(top_emp_mc['Total MC (Days)']):
        ax.text(v + 0.2, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(plt.gcf())
    plt.close()

    # 5. Average MC per Visit by Employees
    plt.figure(figsize=(12, 6))
    top_emp_avg_mc = patient_df.sort_values('Avg MC per Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_avg_mc, x='Employee Name', y='Avg MC per Visit', hue='Employee Name', 
                     palette='Purples_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Avg MC per Visit ({year})', pad=15)
    ax.set_ylabel('Avg MC per Visit (Days)')
    ax.set_xlabel('Employee')
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_emp_avg_mc['Avg MC per Visit']):
        ax.text(i, v + 0.05, f'{v:.2f}', ha='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(plt.gcf())
    plt.close()

    # 6. Division-wise Claim Distribution (Pie)
    plt.figure(figsize=(10, 6))
    division_claims = patient_df.groupby('Division/Department')['Total Claim (Combined)'].sum()
    plt.pie(division_claims, labels=division_claims.index, autopct='%1.1f%%', colors=sns.color_palette('muted'), 
            startangle=90, textprops={'fontsize': 11, 'color': '#333333'}, wedgeprops={'edgecolor': 'black', 'linewidth': 0.5})
    plt.title(f'Claim Distribution by Division ({year})', pad=15)
    plt.tight_layout()
    employee_charts.append(plt.gcf())
    plt.close()

    provider_images = [fig_to_image(fig) if fig is not None else None for fig in provider_charts]
    employee_images = [fig_to_image(fig) for fig in employee_charts]
    return provider_images, employee_images

def fig_to_image(fig):
    if fig is None:
        return None
    fig.canvas.draw()
    img_array = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
    img_array = img_array.reshape(fig.canvas.get_width_height()[::-1] + (4,))
    plt.close(fig)
    return img_array

# --- Gradio Interface ---
with gr.Blocks(title="Claims Analysis Dashboard", css="""
    body { background-color: #f5f6f5; }
    h1, h2 { color: #333333; font-family: Helvetica; }
""") as demo:
    gr.Markdown("# Claims Analysis Dashboard (2024 - Present)")
    
    with gr.Row():
        url_input = gr.Textbox(label="Website URL", placeholder="Enter URL here", lines=1)
        user_id_input = gr.Textbox(label="User ID", placeholder="Enter User ID", lines=1)
        password_input = gr.Textbox(label="Password", type="password", placeholder="Enter Password", lines=1)
    scrape_btn = gr.Button("Submit", variant="primary")
    
    with gr.Row():
        year_dropdown = gr.Dropdown(
            label="Select Year to View Data",
            choices=["2024", "2025"],
            value="2024",
            allow_custom_value=False
        )
        show_mc_pct_checkbox = gr.Checkbox(label="Show % MC Given Chart", value=True)
        mc_sort_dropdown = gr.Dropdown(
            label="Sort % MC Given",
            choices=["desc", "asc"],
            value="desc",
            allow_custom_value=False
        )
    
    status_output = gr.Textbox(label="Status", lines=2, interactive=False)
    patient_state = gr.State()
    claim_state = gr.State()
    mc_state = gr.State()

    with gr.Tabs():
        with gr.TabItem("Provider Insights"):
            gr.Markdown("## Provider Insights Dashboard")
            with gr.Row():
                prov_chart1 = gr.Image(label="Total Visits by Providers", interactive=False)
                prov_chart2 = gr.Image(label="Total MC by Providers", interactive=False)
            with gr.Row():
                prov_chart3 = gr.Image(label="% MC Given by Providers", interactive=False, visible=True)
            with gr.Row():
                prov_chart4 = gr.Image(label="Total Claim by Providers", interactive=False)
                prov_chart5 = gr.Image(label="Average Claim per Visit by Providers", interactive=False)

        with gr.TabItem("Employee Insights"):
            gr.Markdown("## Employee Insights Dashboard")
            with gr.Row():
                emp_chart1 = gr.Image(label="Total Visits by Employees", interactive=False)
                emp_chart2 = gr.Image(label="Total Claim by Employees", interactive=False)
            with gr.Row():
                emp_chart3 = gr.Image(label="Average Claim per Visit by Employees", interactive=False)
                emp_chart4 = gr.Image(label="Total MC by Employees", interactive=False)
            with gr.Row():
                emp_chart5 = gr.Image(label="Average MC per Visit by Employees", interactive=False)
                emp_chart6 = gr.Image(label="Claim Distribution by Division", interactive=False)

    # [Event handlers remain unchanged, just ensure inputs/outputs match the above components]
    def scrape_and_store(url, user_id, password, show_mc_pct, mc_sort_order):
        patient_data_by_year, claim_data_by_year, mc_data_by_year, status = scrape_data(url, user_id, password)
        if patient_data_by_year is None or claim_data_by_year is None or mc_data_by_year is None:
            return status, None, None, None
        
        provider_images, employee_images = generate_dashboard_charts(
            patient_data_by_year, claim_data_by_year, mc_data_by_year, "2024", show_mc_pct, mc_sort_order)
        return (
            status, patient_data_by_year, claim_data_by_year, mc_data_by_year,
            provider_images[0], provider_images[1], provider_images[2], provider_images[3], provider_images[4],
            employee_images[0], employee_images[1], employee_images[2], employee_images[3], employee_images[4], employee_images[5]
        )

    def update_dashboard(year, patient_data_by_year, claim_data_by_year, mc_data_by_year, show_mc_pct, mc_sort_order):
        if not patient_data_by_year or not claim_data_by_year or not mc_data_by_year:
            return [None] * 11
        provider_images, employee_images = generate_dashboard_charts(
            patient_data_by_year, claim_data_by_year, mc_data_by_year, year, show_mc_pct, mc_sort_order)
        return (
            provider_images[0], provider_images[1], provider_images[2], provider_images[3], provider_images[4],
            employee_images[0], employee_images[1], employee_images[2], employee_images[3], employee_images[4], employee_images[5]
        )

    scrape_btn.click(
        fn=lambda url, user_id, password: scrape_and_store(url, user_id, password, show_mc_pct_checkbox.value, mc_sort_dropdown.value),
        inputs=[url_input, user_id_input, password_input],
        outputs=[
            status_output, patient_state, claim_state, mc_state,
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6
        ]
    )

    year_dropdown.change(
        fn=update_dashboard,
        inputs=[year_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=[
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6
        ]
    )

    show_mc_pct_checkbox.change(
        fn=update_dashboard,
        inputs=[year_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=[
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6
        ]
    )

    mc_sort_dropdown.change(
        fn=update_dashboard,
        inputs=[year_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=[
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6
        ]
    )

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://b10ebc6497d93c96c3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "C:\Users\DELL-INTERN-HR\anaconda3\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\DELL-INTERN-HR\anaconda3\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\DELL-INTERN-HR\anaconda3\Lib\site-packages\gradio\blocks.py", line 2108, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\DELL-INTERN-HR\anaconda3\Lib\site-packages\gradio\blocks.py", line 1864, in postprocess_data
    self.validate_outputs(block_fn, predictions)  # type: ignore
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\DELL-INTERN-HR\anaconda3\Lib\site-packag

In [6]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from datetime import datetime
import time
import numpy as np
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException
from io import BytesIO
from pptx import Presentation
from pptx.util import Inches
import tempfile

# --- Scraping Functions ---
def wait_for_element(driver, locator):
    return WebDriverWait(driver, 10).until(EC.element_to_be_clickable(locator))

def select_date_month_day(driver, date_str, date_input_id):
    date_to_select = datetime.strptime(date_str, '%Y-%m-%d')
    date_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, date_input_id)))
    date_input.click()
    month_select = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-datepicker-month')))
    month_option = month_select.find_element(By.XPATH, f"//option[@value='{date_to_select.month - 1}']")
    month_option.click()
    day = date_to_select.day
    day_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//td[@data-handler='selectDay']/a[text()='{day}']")))
    day_element.click()
    time.sleep(5)

def select_date(driver, date_str, date_input_id):
    date_to_select = datetime.strptime(date_str, '%Y-%m-%d')
    date_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, date_input_id)))
    date_input.click()
    time.sleep(2)
    month_select = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-datepicker-month')))
    month_option = month_select.find_element(By.XPATH, f"//option[@value='{date_to_select.month - 1}']")
    time.sleep(2)
    month_option.click()
    year_select = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-datepicker-year')))
    year_option = year_select.find_element(By.XPATH, f"//option[@value='{date_to_select.year}']")
    year_option.click()
    day = date_to_select.day
    day_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//td[@data-handler='selectDay']/a[text()='{day}']")))
    day_element.click()
    time.sleep(5)

def extract_grid_data_clm_summary(driver):
    data = []
    try:
        total_pages_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "sp_1_pjqgridClmSummbyProv")))
        total_pages = int(total_pages_element.text.strip())
    except:
        return data
    for current_page in range(1, total_pages + 1):
        time.sleep(3)
        driver.execute_script("window.scrollTo(0, 0);")
        grid = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "jqgridClmSummbyProv")))
        rows = WebDriverWait(grid, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "tr.jqgrow")))
        for row in rows:
            try:
                provider_name = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_ProvName']").text
                visits = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_NoOfVisit']").text
                claim = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_ClmAmt']").text
                try:
                    total_mc = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_TotalMC']").text
                except:
                    total_mc = '0'
                data.append({'Provider Name': provider_name, 'No of Visits': visits, 'Total Claim': claim, 'Total MC (Days)': total_mc})
            except Exception as e:
                print(f"Error extracting row: {e}")
                continue
        if current_page < total_pages:
            try:
                next_button_div = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn.btn-sm.btn-default span.fa.fa-forward")))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button_div)
                next_button_div.click()
                WebDriverWait(driver, 10).until(EC.staleness_of(rows[0]))
            except:
                break
    return data

def extract_grid_data_patient_analysis(driver):
    all_data = []
    while True:
        driver.execute_script("window.scrollTo(0, 0);")
        grid = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "jqgridCorpMcAnalysis")))
        rows = grid.find_elements(By.CSS_SELECTOR, "tr.jqgrow")
        for row in rows:
            try:
                employee_name = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_MEM_NAME']").text
                employee_no = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_MEM_EMPID']").text
                division = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_MEM_EMPDIVISION']").text
                total_visit = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalVisit']").text
                total_mc = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalMC']").text
                total_claim_own = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalClaim_Own']").text
                total_claim_dep = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalClaim_Dep']").text
                all_data.append({
                    'Employee Name': employee_name, 'Employee No': employee_no, 'Division/Department': division,
                    'Total Visit': total_visit, 'Total MC (Days)': total_mc, 'Total Claim (Own)': total_claim_own,
                    'Total Claim (Dep)': total_claim_dep
                })
            except:
                continue
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            next_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn.btn-sm.btn-default span.fa.fa-forward")))
            parent_div = next_button.find_element(By.XPATH, "./parent::div")
            if "disabled" in parent_div.get_attribute("class"):
                break
            driver.execute_script("arguments[0].scrollIntoView(true);", parent_div)
            parent_div.click()
            WebDriverWait(driver, 10).until(EC.staleness_of(rows[0]))
        except:
            break
    return all_data

def extract_grid_data_mc(driver):
    data = []
    try:
        total_pages_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "sp_1_jqgrid")))
        total_pages = int(total_pages_element.text.strip())
    except:
        total_pages = 1
    for current_page in range(1, total_pages + 1):
        time.sleep(3)
        driver.execute_script("window.scrollTo(0, 0);")
        grid = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "jqgrid")))
        rows = WebDriverWait(grid, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "tr.jqgrow")))
        for row in rows:
            try:
                provider_name = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgrid_STR_ProvName']").text.strip()
                total_mc_given = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgrid_STR_MC_Given_Count']").text.strip()
                total_visits = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgrid_STR_VISITCount']").text.strip()
                data.append({'Provider': provider_name, 'Total MC Given': total_mc_given, 'No. of Visit': total_visits})
            except:
                continue
        if current_page < total_pages:
            try:
                next_button_div = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn.btn-sm.btn-default span.fa.fa-forward")))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button_div)
                next_button_div.click()
                WebDriverWait(driver, 10).until(EC.staleness_of(rows[0]))
            except:
                break
    return data

def scrape_data(url, user_id, password):
    edge_options = Options()
    driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()), options=edge_options)
    start_year = 2024
    current_date = datetime.now().strftime('%Y-%m-%d')
    patient_data_by_year = {}
    claim_data_by_year = {}
    mc_data_by_year = {}
    try:
        driver.get(url)
        image = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//img[@src='/ClaimEXMVR/Servlet_LoadImage?SFC=loadImage&imageName=icorporate.png']")))
        image.click()
        user_id_field = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.NAME, "txtloginid")))
        user_id_field.send_keys(user_id)
        password_field = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "inputpss")))
        password_field.send_keys(password)
        sign_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn.btn-primary[type='submit']")))
        sign_button.click()
        continue_button = wait_for_element(driver, (By.XPATH, "//button[text()='Continue']"))
        continue_button.click()

        for year in range(start_year, 2025 + 1):
            if year == 2024:
                start_date = "2024-01-01"
                end_date = "2024-12-31"
            else:
                start_date = "2025-01-01"
                end_date = current_date
            productivity_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[span[text()='Productivity Reports']]")))
            productivity_link.click()
            patient_analysis_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Patient_Analysis_Report'][span[text()=' Patient Analysis Report ']]")))
            patient_analysis_link.click()
            select_date(driver, start_date, "txtStartDate")
            select_date(driver, end_date, "txtEndDate")
            search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
            search_button.click()
            time.sleep(5)
            dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
            driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
            select = Select(dropdown)
            select.select_by_value("100")
            time.sleep(10)
            patient_data = extract_grid_data_patient_analysis(driver)
            patient_df = pd.DataFrame(patient_data)
            numeric_cols_patient = ['Total Visit', 'Total MC (Days)', 'Total Claim (Own)', 'Total Claim (Dep)']
            for col in numeric_cols_patient:
                patient_df[col] = pd.to_numeric(patient_df[col], errors='coerce')
            patient_df['Total Claim (Combined)'] = patient_df['Total Claim (Own)'] + patient_df['Total Claim (Dep)']
            patient_df['Avg Claim per Visit'] = patient_df['Total Claim (Combined)'] / patient_df['Total Visit']
            patient_df['Avg MC per Visit'] = patient_df['Total MC (Days)'] / patient_df['Total Visit']
            patient_df['Avg Claim per MC'] = patient_df['Total Claim (Combined)'] / patient_df['Total MC (Days)']
            patient_data_by_year[year] = patient_df

        for year in range(start_year, 2025 + 1):
            if year == 2024:
                start_date = "2024-01-01"
                end_date = "2024-12-31"
            else:
                start_date = "2025-01-01"
                end_date = current_date
            productivity_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[span[text()='Productivity Reports']]")))
            productivity_link.click()
            mc_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/MC_HealthCare_By_Provider'][span[text()=' MC by Provider ']]")))
            mc_link.click()
            time.sleep(2)
            select_date(driver, start_date, "txtStartDate")
            select_date(driver, end_date, "txtEndDate")
            search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
            search_button.click()
            time.sleep(10)
            dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
            driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
            select = Select(dropdown)
            select.select_by_value("100")
            time.sleep(10)
            mc_data = extract_grid_data_mc(driver)
            mc_df = pd.DataFrame(mc_data)
            numeric_cols_mc = ['Total MC Given', 'No. of Visit']
            for col in numeric_cols_mc:
                mc_df[col] = pd.to_numeric(mc_df[col], errors='coerce')
            mc_df['% MC Given'] = (mc_df['Total MC Given'] / mc_df['No. of Visit']) * 100
            mc_data_by_year[year] = mc_df

        start_date = "2024-01-01"
        end_date = "2025-01-01"
        reg_claims_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[.//span[contains(text(), 'Registration') and contains(text(), 'Claims')]]")))
        reg_claims_link.click()
        providers_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Claim_Summary_by_Provider_Analysis'][span[text()=' Claim Summary by Providers ']]")))
        providers_link.click()
        time.sleep(5)
        select_date_month_day(driver, start_date, "txtFromDate")
        select_date_month_day(driver, end_date, "txtToDate")
        search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
        driver.execute_script("arguments[0].click();", search_button)
        time.sleep(10)
        dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
        driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
        select = Select(dropdown)
        select.select_by_value("100")
        time.sleep(5)
        claim_data_2024 = extract_grid_data_clm_summary(driver)
        claim_df_2024 = pd.DataFrame(claim_data_2024)
        numeric_cols_claim = ['No of Visits', 'Total Claim', 'Total MC (Days)']
        for col in numeric_cols_claim:
            claim_df_2024[col] = pd.to_numeric(claim_df_2024[col], errors='coerce')
        claim_df_2024['Avg Claim per Visit'] = claim_df_2024['Total Claim'] / claim_df_2024['No of Visits']
        claim_data_by_year[2024] = claim_df_2024

        start_date = "2024-12-31"
        end_date = current_date
        reg_claims_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[.//span[contains(text(), 'Registration') and contains(text(), 'Claims')]]")))
        reg_claims_link.click()
        providers_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Claim_Summary_by_Provider_Analysis'][span[text()=' Claim Summary by Providers ']]")))
        providers_link.click()
        time.sleep(5)
        select_date_month_day(driver, start_date, "txtFromDate")
        select_date_month_day(driver, end_date, "txtToDate")
        search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
        driver.execute_script("arguments[0].click();", search_button)
        time.sleep(10)
        dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
        driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
        select = Select(dropdown)
        select.select_by_value("100")
        time.sleep(5)
        claim_data_recent = extract_grid_data_clm_summary(driver)
        claim_df_recent = pd.DataFrame(claim_data_recent)
        for col in numeric_cols_claim:
            claim_df_recent[col] = pd.to_numeric(claim_df_recent[col], errors='coerce')
        claim_df_recent['Avg Claim per Visit'] = claim_df_recent['Total Claim'] / claim_df_recent['No of Visits']
        claim_data_by_year[2025] = claim_df_recent

        return patient_data_by_year, claim_data_by_year, mc_data_by_year, "Data scraped successfully!"
    except Exception as e:
        return None, None, None, f"Error: {str(e)}"
    finally:
        driver.quit()

# --- Plotting Functions ---
def generate_dashboard_charts(patient_data_by_year, claim_data_by_year, mc_data_by_year, year, show_mc_pct=True, mc_sort_order="desc"):
    if not patient_data_by_year or not claim_data_by_year or not mc_data_by_year:
        print("No data available to generate charts.")
        return None, None, None, None
    
    year_int = int(year)
    patient_df = patient_data_by_year.get(year_int, pd.DataFrame())
    claim_df = claim_data_by_year.get(year_int, pd.DataFrame())
    mc_df = mc_data_by_year.get(year_int, pd.DataFrame())
    
    if patient_df.empty or claim_df.empty or mc_df.empty:
        print(f"Data for year {year} is empty: Patient: {patient_df.empty}, Claim: {claim_df.empty}, MC: {mc_df.empty}")
        return None, None, None, None
    
    sns.set(style="whitegrid", palette="muted")
    plt.rcParams.update({
        'font.family': 'Arial', 'font.size': 12, 'axes.titlesize': 16, 
        'axes.labelsize': 14, 'xtick.labelsize': 11, 'ytick.labelsize': 11,
        'axes.titleweight': 'bold', 'axes.linewidth': 1.5, 'grid.linestyle': ':', 
        'grid.alpha': 0.5, 'figure.facecolor': '#f5f6f5', 'axes.facecolor': '#ffffff',
        'axes.edgecolor': '#333333', 'axes.labelcolor': '#333333', 'text.color': '#333333'
    })
    
    provider_charts = []
    employee_charts = []

    # Provider Charts
    fig = plt.figure(figsize=(12, 6))
    top_prov_visits = mc_df.sort_values('No. of Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_visits, x='No. of Visit', y='Provider', hue='Provider', palette='Blues_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Total Visits ({year})', pad=15)
    ax.set_xlabel('Total Visits')
    ax.set_ylabel('Provider')
    for i, v in enumerate(top_prov_visits['No. of Visit']):
        ax.text(v + 0.5, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_prov_mc = mc_df.sort_values('Total MC Given', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_mc, x='Total MC Given', y='Provider', hue='Provider', palette='Greens_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Total MC Given ({year})', pad=15)
    ax.set_xlabel('Total MC (Days)')
    ax.set_ylabel('Provider')
    for i, v in enumerate(top_prov_mc['Total MC Given']):
        ax.text(v + 0.5, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(fig)
    plt.close(fig)

    if show_mc_pct:
        fig = plt.figure(figsize=(18, 9))
        top_visits_provs = set(mc_df.sort_values('No. of Visit', ascending=False).head(10)['Provider'])
        top_mc_provs = set(mc_df.sort_values('Total MC Given', ascending=False).head(10)['Provider'])
        top_provs = top_visits_provs.union(top_mc_provs)
        top_prov_mc_pct = mc_df[mc_df['Provider'].isin(top_provs)].sort_values(
            '% MC Given', ascending=(mc_sort_order == "asc")).head(20)
        ax = sns.barplot(data=top_prov_mc_pct, x='Provider', y='% MC Given', hue='Provider', 
                         palette='Purples_r', legend=False, edgecolor='black', linewidth=0.5)
        ax.set_title(f'Top 20 Providers by % MC Given ({year}) - Sorted {"Ascending" if mc_sort_order == "asc" else "Descending"}', pad=15)
        ax.set_ylabel('% MC Given', fontsize=14)
        ax.set_xlabel('Provider', fontsize=14)
        plt.xticks(rotation=45, ha='right', fontsize=11)
        for i, v in enumerate(top_prov_mc_pct['% MC Given']):
            ax.text(i, v + 1, f'{v:.1f}%', ha='center', fontsize=10, color='#333333')
        plt.tight_layout()
        provider_charts.append(fig)
    else:
        provider_charts.append(None)
    if show_mc_pct:
        plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_prov_claim = claim_df.sort_values('Total Claim', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_claim, x='Total Claim', y='Provider Name', hue='Provider Name', 
                     palette='Oranges_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Total Claim ({year})', pad=15)
    ax.set_xlabel('Total Claim ($)')
    ax.set_ylabel('Provider')
    for i, v in enumerate(top_prov_claim['Total Claim']):
        ax.text(v + 0.5, i, f'{v:,.2f}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_prov_avg_claim = claim_df.sort_values('Avg Claim per Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_avg_claim, x='Provider Name', y='Avg Claim per Visit', hue='Provider Name', 
                     palette='Reds_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Avg Claim per Visit ({year})', pad=15)
    ax.set_ylabel('Avg Claim per Visit ($)')
    ax.set_xlabel('Provider')
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_prov_avg_claim['Avg Claim per Visit']):
        ax.text(i, v + 0.5, f'{v:.2f}', ha='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(fig)
    plt.close(fig)

    # Employee Charts
    fig = plt.figure(figsize=(12, 6))
    top_emp_visits = patient_df.sort_values('Total Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_visits, x='Total Visit', y='Employee Name', hue='Employee Name', 
                     palette='Blues_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Total Visits ({year})', pad=15)
    ax.set_xlabel('Total Visits')
    ax.set_ylabel('Employee')
    for i, v in enumerate(top_emp_visits['Total Visit']):
        ax.text(v + 0.2, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_emp_claim = patient_df.sort_values('Total Claim (Combined)', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_claim, x='Total Claim (Combined)', y='Employee Name', hue='Employee Name', 
                     palette='Oranges_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Total Claim ({year})', pad=15)
    ax.set_xlabel('Total Claim ($)')
    ax.set_ylabel('Employee')
    for i, v in enumerate(top_emp_claim['Total Claim (Combined)']):
        ax.text(v + 1, i, f'{v:,.2f}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_emp_avg_claim = patient_df.sort_values('Avg Claim per Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_avg_claim, x='Employee Name', y='Avg Claim per Visit', hue='Employee Name', 
                     palette='Reds_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Avg Claim per Visit ({year})', pad=15)
    ax.set_ylabel('Avg Claim per Visit ($)')
    ax.set_xlabel('Employee')
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_emp_avg_claim['Avg Claim per Visit']):
        ax.text(i, v + 0.5, f'{v:.2f}', ha='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_emp_mc = patient_df.sort_values('Total MC (Days)', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_mc, x='Total MC (Days)', y='Employee Name', hue='Employee Name', 
                     palette='Greens_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Total MC ({year})', pad=15)
    ax.set_xlabel('Total MC (Days)')
    ax.set_ylabel('Employee')
    for i, v in enumerate(top_emp_mc['Total MC (Days)']):
        ax.text(v + 0.2, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_emp_avg_mc = patient_df.sort_values('Avg MC per Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_avg_mc, x='Employee Name', y='Avg MC per Visit', hue='Employee Name', 
                     palette='Purples_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Avg MC per Visit ({year})', pad=15)
    ax.set_ylabel('Avg MC per Visit (Days)')
    ax.set_xlabel('Employee')
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_emp_avg_mc['Avg MC per Visit']):
        ax.text(i, v + 0.05, f'{v:.2f}', ha='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(10, 6))
    division_claims = patient_df.groupby('Division/Department')['Total Claim (Combined)'].sum()
    plt.pie(division_claims, labels=division_claims.index, autopct='%1.1f%%', colors=sns.color_palette('muted'), 
            startangle=90, textprops={'fontsize': 11, 'color': '#333333'}, wedgeprops={'edgecolor': 'black', 'linewidth': 0.5})
    plt.title(f'Claim Distribution by Division ({year})', pad=15)
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    provider_images = [fig_to_image(fig) if fig is not None else None for fig in provider_charts]
    employee_images = [fig_to_image(fig) for fig in employee_charts]
    
    print(f"Generated {len(provider_charts)} provider charts and {len(employee_charts)} employee charts for year {year}")
    return provider_images, employee_images, provider_charts, employee_charts

def fig_to_image(fig):
    if fig is None:
        return None
    fig.canvas.draw()
    img_array = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
    img_array = img_array.reshape(fig.canvas.get_width_height()[::-1] + (4,))
    return img_array

def charts_to_pptx(provider_charts, employee_charts, year):
    try:
        prs = Presentation()
        slide_layout = prs.slide_layouts[5]  # Blank slide layout with title
        prs.slide_width = Inches(13.33)  # Widescreen slide size
        prs.slide_height = Inches(7.5)

        # Define chart titles
        provider_titles = [
            f"Top 10 Providers by Total Visits ({year})",
            f"Top 10 Providers by Total MC Given ({year})",
            f"Top 20 Providers by % MC Given ({year})",
            f"Top 10 Providers by Total Claim ({year})",
            f"Top 10 Providers by Avg Claim per Visit ({year})"
        ]
        
        employee_titles = [
            f"Top 10 Employees by Total Visits ({year})",
            f"Top 10 Employees by Total Claim ({year})",
            f"Top 10 Employees by Avg Claim per Visit ({year})",
            f"Top 10 Employees by Total MC ({year})",
            f"Top 10 Employees by Avg MC per Visit ({year})",
            f"Claim Distribution by Division ({year})"
        ]

        # Provider Slide (all charts on one slide)
        if provider_charts:
            slide = prs.slides.add_slide(slide_layout)
            slide.shapes.title.text = f"Provider Insights ({year})"
            num_charts = sum(1 for fig in provider_charts if fig is not None)
            if num_charts == 0:
                print("No valid provider charts to export")
            else:
                # Arrange charts in a grid: up to 3 per row, adjust height dynamically
                charts_per_row = 3
                rows = (num_charts + charts_per_row - 1) // charts_per_row
                chart_width = Inches(3.8)  # Reduced width to fit 3 charts
                chart_height = Inches(2.5) if rows > 1 else Inches(5)  # Adjust height based on rows
                top_start = Inches(1.5)  # Space below title
                valid_charts = [fig for fig in provider_charts if fig is not None]
                
                for idx, fig in enumerate(valid_charts):
                    row = idx // charts_per_row
                    col = idx % charts_per_row
                    left = Inches(0.5 + col * 4.5)
                    top = top_start + row * (chart_height + Inches(0.2))  # Small gap between rows
                    img_stream = BytesIO()
                    fig.savefig(img_stream, format='png', bbox_inches='tight', dpi=100)  # Lower DPI for smaller size
                    img_stream.seek(0)
                    if img_stream.getvalue():
                        slide.shapes.add_picture(img_stream, left, top, width=chart_width, height=chart_height)
                        print(f"Added chart {idx+1}: '{provider_titles[idx]}' to Provider slide")
                    else:
                        print(f"Failed to save chart {idx+1}: '{provider_titles[idx]}' - empty image stream")

        # Employee Slide (all charts on one slide)
        if employee_charts:
            slide = prs.slides.add_slide(slide_layout)
            slide.shapes.title.text = f"Employee Insights ({year})"
            num_charts = sum(1 for fig in employee_charts if fig is not None)
            if num_charts == 0:
                print("No valid employee charts to export")
            else:
                # Arrange charts in a grid: up to 3 per row
                charts_per_row = 3
                rows = (num_charts + charts_per_row - 1) // charts_per_row
                chart_width = Inches(3.8)  # Reduced width to fit 3 charts
                chart_height = Inches(2.5) if rows > 1 else Inches(5)  # Adjust height based on rows
                top_start = Inches(1.5)  # Space below title
                valid_charts = [fig for fig in employee_charts if fig is not None]
                
                for idx, fig in enumerate(valid_charts):
                    row = idx // charts_per_row
                    col = idx % charts_per_row
                    left = Inches(0.5 + col * 4.5)
                    top = top_start + row * (chart_height + Inches(0.2))  # Small gap between rows
                    img_stream = BytesIO()
                    fig.savefig(img_stream, format='png', bbox_inches='tight', dpi=100)  # Lower DPI for smaller size
                    img_stream.seek(0)
                    if img_stream.getvalue():
                        slide.shapes.add_picture(img_stream, left, top, width=chart_width, height=chart_height)
                        print(f"Added chart {idx+1}: '{employee_titles[idx]}' to Employee slide")
                    else:
                        print(f"Failed to save chart {idx+1}: '{employee_titles[idx]}' - empty image stream")

        with tempfile.NamedTemporaryFile(delete=False, suffix='.pptx') as tmp_file:
            prs.save(tmp_file.name)
            tmp_file_path = tmp_file.name

        print(f"PPTX file saved to temporary path: {tmp_file_path}")
        return tmp_file_path

    except Exception as e:
        print(f"Error generating PPTX: {str(e)}")
        return None

# --- Gradio Interface ---
with gr.Blocks(title="Claims Analysis Dashboard", css="""
    body { background-color: #f5f6f5; }
    h1, h2 { color: #333333; font-family: Arial; }
""") as demo:
    gr.Markdown("# Claims Analysis Dashboard (2024 - Present)")
    
    with gr.Row():
        url_input = gr.Textbox(label="Website URL", placeholder="Enter URL here", lines=1)
        user_id_input = gr.Textbox(label="User ID", placeholder="Enter User ID", lines=1)
        password_input = gr.Textbox(label="Password", type="password", placeholder="Enter Password", lines=1)
    scrape_btn = gr.Button("Submit", variant="primary")
    
    with gr.Row():
        year_dropdown = gr.Dropdown(
            label="Select Year to View Data",
            choices=["2024"],  # Default choice to avoid empty list
            value="2024",      # Default value matching the choice
            allow_custom_value=False,
            interactive=False  # Initially non-interactive until data is scraped
        )
        show_mc_pct_checkbox = gr.Checkbox(label="Show % MC Given Chart", value=True)
        mc_sort_dropdown = gr.Dropdown(
            label="Sort % MC Given",
            choices=["desc", "asc"],
            value="desc",
            allow_custom_value=False
        )
    
    status_output = gr.Textbox(label="Status", lines=2, interactive=False)
    patient_state = gr.State()
    claim_state = gr.State()
    mc_state = gr.State()

    with gr.Tabs():
        with gr.TabItem("Provider Insights"):
            gr.Markdown("## Provider Insights Dashboard")
            with gr.Row():
                prov_chart1 = gr.Image(label="Total Visits by Providers", interactive=False)
                prov_chart2 = gr.Image(label="Total MC by Providers", interactive=False)
            with gr.Row():
                prov_chart3 = gr.Image(label="% MC Given by Providers", interactive=False, visible=True)
            with gr.Row():
                prov_chart4 = gr.Image(label="Total Claim by Providers", interactive=False)
                prov_chart5 = gr.Image(label="Average Claim per Visit by Providers", interactive=False)
            download_btn_prov = gr.Button("Download Provider Charts as PPTX")

        with gr.TabItem("Employee Insights"):
            gr.Markdown("## Employee Insights Dashboard")
            with gr.Row():
                emp_chart1 = gr.Image(label="Total Visits by Employees", interactive=False)
                emp_chart2 = gr.Image(label="Total Claim by Employees", interactive=False)
            with gr.Row():
                emp_chart3 = gr.Image(label="Average Claim per Visit by Employees", interactive=False)
                emp_chart4 = gr.Image(label="Total MC by Employees", interactive=False)
            with gr.Row():
                emp_chart5 = gr.Image(label="Average MC per Visit by Employees", interactive=False)
                emp_chart6 = gr.Image(label="Claim Distribution by Division", interactive=False)
            download_btn_emp = gr.Button("Download Employee Charts as PPTX")

    def scrape_and_store(url, user_id, password, show_mc_pct, mc_sort_order):
        patient_data_by_year, claim_data_by_year, mc_data_by_year, status = scrape_data(url, user_id, password)
        if patient_data_by_year is None or claim_data_by_year is None or mc_data_by_year is None:
            return (
                status, None, None, None, 
                gr.update(choices=["2024"], value="2024", interactive=False),  
                None, None, None, None, None, None, None, None, None, None, None
            )

        available_years = sorted(set(patient_data_by_year.keys()) | set(claim_data_by_year.keys()) | set(mc_data_by_year.keys()))
        year_choices = [str(year) for year in available_years]
        default_year = year_choices[0] if year_choices else "2024"

        provider_images, employee_images, provider_figs, employee_figs = generate_dashboard_charts(
            patient_data_by_year, claim_data_by_year, mc_data_by_year, default_year, show_mc_pct, mc_sort_order)
        return (
            status, patient_data_by_year, claim_data_by_year, mc_data_by_year,
            gr.update(choices=year_choices, value=default_year, interactive=True),  
            provider_images[0], provider_images[1], provider_images[2], provider_images[3], provider_images[4],
            employee_images[0], employee_images[1], employee_images[2], employee_images[3], employee_images[4], employee_images[5]
        )

    def update_dashboard(year, patient_data_by_year, claim_data_by_year, mc_data_by_year, show_mc_pct, mc_sort_order):
        if not patient_data_by_year or not claim_data_by_year or not mc_data_by_year:
            return [None] * 11
        provider_images, employee_images, _, _ = generate_dashboard_charts(
            patient_data_by_year, claim_data_by_year, mc_data_by_year, year, show_mc_pct, mc_sort_order)
        return (
            provider_images[0], provider_images[1], provider_images[2], provider_images[3], provider_images[4],
            employee_images[0], employee_images[1], employee_images[2], employee_images[3], employee_images[4], employee_images[5]
        )

    def download_provider_pptx(year, patient_data_by_year, claim_data_by_year, mc_data_by_year, show_mc_pct, mc_sort_order):
        _, _, provider_figs, _ = generate_dashboard_charts(
            patient_data_by_year, claim_data_by_year, mc_data_by_year, year, show_mc_pct, mc_sort_order)
        if not provider_figs:
            print("No provider figures to export")
            return None
        pptx_path = charts_to_pptx(provider_figs, [], year)
        if pptx_path:
            return gr.File(value=pptx_path, label=f"Provider_Charts_{year}.pptx")
        else:
            print("Failed to generate provider PPTX")
            return None

    def download_employee_pptx(year, patient_data_by_year, claim_data_by_year, mc_data_by_year, show_mc_pct, mc_sort_order):
        _, _, _, employee_figs = generate_dashboard_charts(
            patient_data_by_year, claim_data_by_year, mc_data_by_year, year, show_mc_pct, mc_sort_order)
        if not employee_figs:
            print("No employee figures to export")
            return None
        pptx_path = charts_to_pptx([], employee_figs, year)
        if pptx_path:
            return gr.File(value=pptx_path, label=f"Employee_Charts_{year}.pptx")
        else:
            print("Failed to generate employee PPTX")
            return None

    scrape_btn.click(
        fn=scrape_and_store,
        inputs=[url_input, user_id_input, password_input, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=[
            status_output, patient_state, claim_state, mc_state, year_dropdown,
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6
        ]
    )

    year_dropdown.change(
        fn=update_dashboard,
        inputs=[year_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=[
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6
        ]
    )

    show_mc_pct_checkbox.change(
        fn=update_dashboard,
        inputs=[year_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=[
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6
        ]
    )

    mc_sort_dropdown.change(
        fn=update_dashboard,
        inputs=[year_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=[
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6
        ]
    )

    download_btn_prov.click(
        fn=download_provider_pptx,
        inputs=[year_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=gr.File()
    )

    download_btn_emp.click(
        fn=download_employee_pptx,
        inputs=[year_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=gr.File()
    )

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7866
* Running on public URL: https://8097eaa4b2f624d670.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Generated 5 provider charts and 6 employee charts for year 2024
Generated 5 provider charts and 6 employee charts for year 2024
Added chart 1: 'Top 10 Providers by Total Visits (2024)' to Provider slide
Added chart 2: 'Top 10 Providers by Total MC Given (2024)' to Provider slide
Added chart 3: 'Top 20 Providers by % MC Given (2024)' to Provider slide
Added chart 4: 'Top 10 Providers by Total Claim (2024)' to Provider slide
Added chart 5: 'Top 10 Providers by Avg Claim per Visit (2024)' to Provider slide
PPTX file saved to temporary path: C:\Users\DELL-I~1\AppData\Local\Temp\tmpchw8yion.pptx
Generated 5 provider charts and 6 employee charts for year 2024
Added chart 1: 'Top 10 Employees by Total Visits (2024)' to Employee slide
Added chart 2: 'Top 10 Employees by Total Claim (2024)' to Employee slide
Added chart 3: 'Top 10 Employees by Avg Claim per Visit (2024)' to Employee slide
Added chart 4: 'Top 10 Employees by Total MC (2024)' to Employee slide
Added chart 5: 'Top 10 Employees by 

In [7]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from datetime import datetime
import time
import numpy as np
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException
from io import BytesIO
from pptx import Presentation
from pptx.util import Inches
import tempfile

# --- Scraping Functions ---
def wait_for_element(driver, locator):
    return WebDriverWait(driver, 10).until(EC.element_to_be_clickable(locator))

def select_date_month_day(driver, date_str, date_input_id):
    date_to_select = datetime.strptime(date_str, '%Y-%m-%d')
    date_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, date_input_id)))
    date_input.click()
    month_select = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-datepicker-month')))
    month_option = month_select.find_element(By.XPATH, f"//option[@value='{date_to_select.month - 1}']")
    month_option.click()
    day = date_to_select.day
    day_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//td[@data-handler='selectDay']/a[text()='{day}']")))
    day_element.click()
    time.sleep(5)

def select_date(driver, date_str, date_input_id):
    date_to_select = datetime.strptime(date_str, '%Y-%m-%d')
    date_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, date_input_id)))
    date_input.click()
    time.sleep(2)
    month_select = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-datepicker-month')))
    month_option = month_select.find_element(By.XPATH, f"//option[@value='{date_to_select.month - 1}']")
    time.sleep(2)
    month_option.click()
    year_select = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-datepicker-year')))
    year_option = year_select.find_element(By.XPATH, f"//option[@value='{date_to_select.year}']")
    year_option.click()
    day = date_to_select.day
    day_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//td[@data-handler='selectDay']/a[text()='{day}']")))
    day_element.click()
    time.sleep(5)

def extract_grid_data_clm_summary(driver):
    data = []
    try:
        total_pages_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "sp_1_pjqgridClmSummbyProv")))
        total_pages = int(total_pages_element.text.strip())
    except:
        return data
    for current_page in range(1, total_pages + 1):
        time.sleep(3)
        driver.execute_script("window.scrollTo(0, 0);")
        grid = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "jqgridClmSummbyProv")))
        rows = WebDriverWait(grid, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "tr.jqgrow")))
        for row in rows:
            try:
                provider_name = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_ProvName']").text
                visits = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_NoOfVisit']").text
                claim = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_ClmAmt']").text
                try:
                    total_mc = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_TotalMC']").text
                except:
                    total_mc = '0'
                data.append({'Provider Name': provider_name, 'No of Visits': visits, 'Total Claim': claim, 'Total MC (Days)': total_mc})
            except Exception as e:
                print(f"Error extracting row: {e}")
                continue
        if current_page < total_pages:
            try:
                next_button_div = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn.btn-sm.btn-default span.fa.fa-forward")))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button_div)
                next_button_div.click()
                WebDriverWait(driver, 10).until(EC.staleness_of(rows[0]))
            except:
                break
    return data

def extract_grid_data_patient_analysis(driver):
    all_data = []
    while True:
        driver.execute_script("window.scrollTo(0, 0);")
        grid = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "jqgridCorpMcAnalysis")))
        rows = grid.find_elements(By.CSS_SELECTOR, "tr.jqgrow")
        for row in rows:
            try:
                employee_name = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_MEM_NAME']").text
                employee_no = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_MEM_EMPID']").text
                division = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_MEM_EMPDIVISION']").text
                total_visit = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalVisit']").text
                total_mc = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalMC']").text
                total_claim_own = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalClaim_Own']").text
                total_claim_dep = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalClaim_Dep']").text
                all_data.append({
                    'Employee Name': employee_name, 'Employee No': employee_no, 'Division/Department': division,
                    'Total Visit': total_visit, 'Total MC (Days)': total_mc, 'Total Claim (Own)': total_claim_own,
                    'Total Claim (Dep)': total_claim_dep
                })
            except:
                continue
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            next_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn.btn-sm.btn-default span.fa.fa-forward")))
            parent_div = next_button.find_element(By.XPATH, "./parent::div")
            if "disabled" in parent_div.get_attribute("class"):
                break
            driver.execute_script("arguments[0].scrollIntoView(true);", parent_div)
            parent_div.click()
            WebDriverWait(driver, 10).until(EC.staleness_of(rows[0]))
        except:
            break
    return all_data

def extract_grid_data_mc(driver):
    data = []
    try:
        total_pages_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "sp_1_jqgrid")))
        total_pages = int(total_pages_element.text.strip())
    except:
        total_pages = 1
    for current_page in range(1, total_pages + 1):
        time.sleep(3)
        driver.execute_script("window.scrollTo(0, 0);")
        grid = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "jqgrid")))
        rows = WebDriverWait(grid, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "tr.jqgrow")))
        for row in rows:
            try:
                provider_name = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgrid_STR_ProvName']").text.strip()
                total_mc_given = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgrid_STR_MC_Given_Count']").text.strip()
                total_visits = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgrid_STR_VISITCount']").text.strip()
                data.append({'Provider': provider_name, 'Total MC Given': total_mc_given, 'No. of Visit': total_visits})
            except:
                continue
        if current_page < total_pages:
            try:
                next_button_div = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn.btn-sm.btn-default span.fa.fa-forward")))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button_div)
                next_button_div.click()
                WebDriverWait(driver, 10).until(EC.staleness_of(rows[0]))
            except:
                break
    return data

def scrape_data(url, user_id, password):
    edge_options = Options()
    driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()), options=edge_options)
    start_year = 2024
    current_date = datetime.now().strftime('%Y-%m-%d')
    patient_data_by_year = {}
    claim_data_by_year = {}
    mc_data_by_year = {}
    try:
        driver.get(url)
        image = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//img[@src='/ClaimEXMVR/Servlet_LoadImage?SFC=loadImage&imageName=icorporate.png']")))
        image.click()
        user_id_field = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.NAME, "txtloginid")))
        user_id_field.send_keys(user_id)
        password_field = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "inputpss")))
        password_field.send_keys(password)
        sign_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn.btn-primary[type='submit']")))
        sign_button.click()
        continue_button = wait_for_element(driver, (By.XPATH, "//button[text()='Continue']"))
        continue_button.click()

        for year in range(start_year, 2025 + 1):
            if year == 2024:
                start_date = "2024-01-01"
                end_date = "2024-12-31"
            else:
                start_date = "2025-01-01"
                end_date = current_date
            productivity_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[span[text()='Productivity Reports']]")))
            productivity_link.click()
            patient_analysis_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Patient_Analysis_Report'][span[text()=' Patient Analysis Report ']]")))
            patient_analysis_link.click()
            select_date(driver, start_date, "txtStartDate")
            select_date(driver, end_date, "txtEndDate")
            search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
            search_button.click()
            time.sleep(5)
            dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
            driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
            select = Select(dropdown)
            select.select_by_value("100")
            time.sleep(10)
            patient_data = extract_grid_data_patient_analysis(driver)
            patient_df = pd.DataFrame(patient_data)
            numeric_cols_patient = ['Total Visit', 'Total MC (Days)', 'Total Claim (Own)', 'Total Claim (Dep)']
            for col in numeric_cols_patient:
                patient_df[col] = pd.to_numeric(patient_df[col], errors='coerce')
            patient_df['Total Claim (Combined)'] = patient_df['Total Claim (Own)'] + patient_df['Total Claim (Dep)']
            patient_df['Avg Claim per Visit'] = patient_df['Total Claim (Combined)'] / patient_df['Total Visit']
            patient_df['Avg MC per Visit'] = patient_df['Total MC (Days)'] / patient_df['Total Visit']
            patient_df['Avg Claim per MC'] = patient_df['Total Claim (Combined)'] / patient_df['Total MC (Days)']
            patient_data_by_year[year] = patient_df

        for year in range(start_year, 2025 + 1):
            if year == 2024:
                start_date = "2024-01-01"
                end_date = "2024-12-31"
            else:
                start_date = "2025-01-01"
                end_date = current_date
            productivity_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[span[text()='Productivity Reports']]")))
            productivity_link.click()
            mc_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/MC_HealthCare_By_Provider'][span[text()=' MC by Provider ']]")))
            mc_link.click()
            time.sleep(2)
            select_date(driver, start_date, "txtStartDate")
            select_date(driver, end_date, "txtEndDate")
            search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
            search_button.click()
            time.sleep(10)
            dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
            driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
            select = Select(dropdown)
            select.select_by_value("100")
            time.sleep(10)
            mc_data = extract_grid_data_mc(driver)
            mc_df = pd.DataFrame(mc_data)
            numeric_cols_mc = ['Total MC Given', 'No. of Visit']
            for col in numeric_cols_mc:
                mc_df[col] = pd.to_numeric(mc_df[col], errors='coerce')
            mc_df['% MC Given'] = (mc_df['Total MC Given'] / mc_df['No. of Visit']) * 100
            mc_data_by_year[year] = mc_df

        start_date = "2024-01-01"
        end_date = "2025-01-01"
        reg_claims_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[.//span[contains(text(), 'Registration') and contains(text(), 'Claims')]]")))
        reg_claims_link.click()
        providers_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Claim_Summary_by_Provider_Analysis'][span[text()=' Claim Summary by Providers ']]")))
        providers_link.click()
        time.sleep(5)
        select_date_month_day(driver, start_date, "txtFromDate")
        select_date_month_day(driver, end_date, "txtToDate")
        search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
        driver.execute_script("arguments[0].click();", search_button)
        time.sleep(10)
        dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
        driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
        select = Select(dropdown)
        select.select_by_value("100")
        time.sleep(5)
        claim_data_2024 = extract_grid_data_clm_summary(driver)
        claim_df_2024 = pd.DataFrame(claim_data_2024)
        numeric_cols_claim = ['No of Visits', 'Total Claim', 'Total MC (Days)']
        for col in numeric_cols_claim:
            claim_df_2024[col] = pd.to_numeric(claim_df_2024[col], errors='coerce')
        claim_df_2024['Avg Claim per Visit'] = claim_df_2024['Total Claim'] / claim_df_2024['No of Visits']
        claim_data_by_year[2024] = claim_df_2024

        start_date = "2024-12-31"
        end_date = current_date
        reg_claims_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[.//span[contains(text(), 'Registration') and contains(text(), 'Claims')]]")))
        reg_claims_link.click()
        providers_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Claim_Summary_by_Provider_Analysis'][span[text()=' Claim Summary by Providers ']]")))
        providers_link.click()
        time.sleep(5)
        select_date_month_day(driver, start_date, "txtFromDate")
        select_date_month_day(driver, end_date, "txtToDate")
        search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
        driver.execute_script("arguments[0].click();", search_button)
        time.sleep(10)
        dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
        driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
        select = Select(dropdown)
        select.select_by_value("100")
        time.sleep(5)
        claim_data_recent = extract_grid_data_clm_summary(driver)
        claim_df_recent = pd.DataFrame(claim_data_recent)
        for col in numeric_cols_claim:
            claim_df_recent[col] = pd.to_numeric(claim_df_recent[col], errors='coerce')
        claim_df_recent['Avg Claim per Visit'] = claim_df_recent['Total Claim'] / claim_df_recent['No of Visits']
        claim_data_by_year[2025] = claim_df_recent

        return patient_data_by_year, claim_data_by_year, mc_data_by_year, "Data scraped successfully!"
    except Exception as e:
        return None, None, None, f"Error: {str(e)}"
    finally:
        driver.quit()

# --- Plotting Functions ---
def generate_dashboard_charts(patient_data_by_year, claim_data_by_year, mc_data_by_year, year, show_mc_pct=True, mc_sort_order="desc"):
    if not patient_data_by_year or not claim_data_by_year or not mc_data_by_year:
        print("No data available to generate charts.")
        return None, None, None, None
    
    year_int = int(year)
    patient_df = patient_data_by_year.get(year_int, pd.DataFrame())
    claim_df = claim_data_by_year.get(year_int, pd.DataFrame())
    mc_df = mc_data_by_year.get(year_int, pd.DataFrame())
    
    if patient_df.empty or claim_df.empty or mc_df.empty:
        print(f"Data for year {year} is empty: Patient: {patient_df.empty}, Claim: {claim_df.empty}, MC: {mc_df.empty}")
        return None, None, None, None
    
    sns.set(style="whitegrid", palette="muted")
    plt.rcParams.update({
        'font.family': 'Arial', 'font.size': 12, 'axes.titlesize': 16, 
        'axes.labelsize': 14, 'xtick.labelsize': 11, 'ytick.labelsize': 11,
        'axes.titleweight': 'bold', 'axes.linewidth': 1.5, 'grid.linestyle': ':', 
        'grid.alpha': 0.5, 'figure.facecolor': '#f5f6f5', 'axes.facecolor': '#ffffff',
        'axes.edgecolor': '#333333', 'axes.labelcolor': '#333333', 'text.color': '#333333'
    })
    
    provider_charts = []
    employee_charts = []

    # Provider Charts
    fig = plt.figure(figsize=(12, 6))
    top_prov_visits = mc_df.sort_values('No. of Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_visits, x='No. of Visit', y='Provider', hue='Provider', palette='Blues_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Total Visits ({year})', pad=15)
    ax.set_xlabel('Total Visits')
    ax.set_ylabel('Provider')
    for i, v in enumerate(top_prov_visits['No. of Visit']):
        ax.text(v + 0.5, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_prov_mc = mc_df.sort_values('Total MC Given', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_mc, x='Total MC Given', y='Provider', hue='Provider', palette='Greens_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Total MC Given ({year})', pad=15)
    ax.set_xlabel('Total MC (Days)')
    ax.set_ylabel('Provider')
    for i, v in enumerate(top_prov_mc['Total MC Given']):
        ax.text(v + 0.5, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(fig)
    plt.close(fig)

    if show_mc_pct:
        fig = plt.figure(figsize=(18, 9))
        top_visits_provs = set(mc_df.sort_values('No. of Visit', ascending=False).head(10)['Provider'])
        top_mc_provs = set(mc_df.sort_values('Total MC Given', ascending=False).head(10)['Provider'])
        top_provs = top_visits_provs.union(top_mc_provs)
        top_prov_mc_pct = mc_df[mc_df['Provider'].isin(top_provs)].sort_values(
            '% MC Given', ascending=(mc_sort_order == "asc")).head(20)
        ax = sns.barplot(data=top_prov_mc_pct, x='Provider', y='% MC Given', hue='Provider', 
                         palette='Purples_r', legend=False, edgecolor='black', linewidth=0.5)
        ax.set_title(f'Top 20 Providers by % MC Given ({year}) - Sorted {"Ascending" if mc_sort_order == "asc" else "Descending"}', pad=15)
        ax.set_ylabel('% MC Given', fontsize=14)
        ax.set_xlabel('Provider', fontsize=14)
        plt.xticks(rotation=45, ha='right', fontsize=11)
        for i, v in enumerate(top_prov_mc_pct['% MC Given']):
            ax.text(i, v + 1, f'{v:.1f}%', ha='center', fontsize=10, color='#333333')
        plt.tight_layout()
        provider_charts.append(fig)
    else:
        provider_charts.append(None)
    if show_mc_pct:
        plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_prov_claim = claim_df.sort_values('Total Claim', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_claim, x='Total Claim', y='Provider Name', hue='Provider Name', 
                     palette='Oranges_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Total Claim ({year})', pad=15)
    ax.set_xlabel('Total Claim ($)')
    ax.set_ylabel('Provider')
    for i, v in enumerate(top_prov_claim['Total Claim']):
        ax.text(v + 0.5, i, f'{v:,.2f}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_prov_avg_claim = claim_df.sort_values('Avg Claim per Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_avg_claim, x='Provider Name', y='Avg Claim per Visit', hue='Provider Name', 
                     palette='Reds_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Avg Claim per Visit ({year})', pad=15)
    ax.set_ylabel('Avg Claim per Visit ($)')
    ax.set_xlabel('Provider')
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_prov_avg_claim['Avg Claim per Visit']):
        ax.text(i, v + 0.5, f'{v:.2f}', ha='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(fig)
    plt.close(fig)

    # Employee Charts
    fig = plt.figure(figsize=(12, 6))
    top_emp_visits = patient_df.sort_values('Total Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_visits, x='Total Visit', y='Employee Name', hue='Employee Name', 
                     palette='Blues_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Total Visits ({year})', pad=15)
    ax.set_xlabel('Total Visits')
    ax.set_ylabel('Employee')
    for i, v in enumerate(top_emp_visits['Total Visit']):
        ax.text(v + 0.2, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_emp_claim = patient_df.sort_values('Total Claim (Combined)', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_claim, x='Total Claim (Combined)', y='Employee Name', hue='Employee Name', 
                     palette='Oranges_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Total Claim ({year})', pad=15)
    ax.set_xlabel('Total Claim ($)')
    ax.set_ylabel('Employee')
    for i, v in enumerate(top_emp_claim['Total Claim (Combined)']):
        ax.text(v + 1, i, f'{v:,.2f}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_emp_avg_claim = patient_df.sort_values('Avg Claim per Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_avg_claim, x='Employee Name', y='Avg Claim per Visit', hue='Employee Name', 
                     palette='Reds_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Avg Claim per Visit ({year})', pad=15)
    ax.set_ylabel('Avg Claim per Visit ($)')
    ax.set_xlabel('Employee')
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_emp_avg_claim['Avg Claim per Visit']):
        ax.text(i, v + 0.5, f'{v:.2f}', ha='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_emp_mc = patient_df.sort_values('Total MC (Days)', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_mc, x='Total MC (Days)', y='Employee Name', hue='Employee Name', 
                     palette='Greens_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Total MC ({year})', pad=15)
    ax.set_xlabel('Total MC (Days)')
    ax.set_ylabel('Employee')
    for i, v in enumerate(top_emp_mc['Total MC (Days)']):
        ax.text(v + 0.2, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_emp_avg_mc = patient_df.sort_values('Avg MC per Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_avg_mc, x='Employee Name', y='Avg MC per Visit', hue='Employee Name', 
                     palette='Purples_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Avg MC per Visit ({year})', pad=15)
    ax.set_ylabel('Avg MC per Visit (Days)')
    ax.set_xlabel('Employee')
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_emp_avg_mc['Avg MC per Visit']):
        ax.text(i, v + 0.05, f'{v:.2f}', ha='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(10, 6))
    division_claims = patient_df.groupby('Division/Department')['Total Claim (Combined)'].sum()
    plt.pie(division_claims, labels=division_claims.index, autopct='%1.1f%%', colors=sns.color_palette('muted'), 
            startangle=90, textprops={'fontsize': 11, 'color': '#333333'}, wedgeprops={'edgecolor': 'black', 'linewidth': 0.5})
    plt.title(f'Claim Distribution by Division ({year})', pad=15)
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    provider_images = [fig_to_image(fig) if fig is not None else None for fig in provider_charts]
    employee_images = [fig_to_image(fig) for fig in employee_charts]
    
    print(f"Generated {len(provider_charts)} provider charts and {len(employee_charts)} employee charts for year {year}")
    return provider_images, employee_images, provider_charts, employee_charts

# --- New Monthly Plotting Function ---
def generate_monthly_dashboard_charts(patient_data_by_year, claim_data_by_year, mc_data_by_year, year, month, show_mc_pct=True, mc_sort_order="desc"):
    if not patient_data_by_year or not claim_data_by_year or not mc_data_by_year:
        print("No data available to generate monthly charts.")
        return None, None, None, None
    
    year_int = int(year)
    month_int = int(month.split('-')[1])  # Extract month number from "YYYY-MM" format
    month_name = datetime.strptime(str(month_int), '%m').strftime('%B')  # Convert to month name
    
    patient_df = patient_data_by_year.get(year_int, pd.DataFrame())
    claim_df = claim_data_by_year.get(year_int, pd.DataFrame())
    mc_df = mc_data_by_year.get(year_int, pd.DataFrame())
    
    if patient_df.empty or claim_df.empty or mc_df.empty:
        print(f"Data for year {year} is empty: Patient: {patient_df.empty}, Claim: {claim_df.empty}, MC: {mc_df.empty}")
        return None, None, None, None
    
    # Since we don't have date-specific data, we'll assume the data is yearly and simulate monthly by dividing
    # For a real implementation, you'd need date columns in your scraped data to filter by month
    patient_df_month = patient_df.copy()  # Placeholder: assumes yearly data split evenly
    claim_df_month = claim_df.copy()
    mc_df_month = mc_df.copy()
    
    sns.set(style="whitegrid", palette="muted")
    plt.rcParams.update({
        'font.family': 'Arial', 'font.size': 12, 'axes.titlesize': 16, 
        'axes.labelsize': 14, 'xtick.labelsize': 11, 'ytick.labelsize': 11,
        'axes.titleweight': 'bold', 'axes.linewidth': 1.5, 'grid.linestyle': ':', 
        'grid.alpha': 0.5, 'figure.facecolor': '#f5f6f5', 'axes.facecolor': '#ffffff',
        'axes.edgecolor': '#333333', 'axes.labelcolor': '#333333', 'text.color': '#333333'
    })
    
    provider_charts = []
    employee_charts = []

    # Provider Charts
    fig = plt.figure(figsize=(12, 6))
    top_prov_visits = mc_df_month.sort_values('No. of Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_visits, x='No. of Visit', y='Provider', hue='Provider', palette='Blues_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Total Visits ({month_name} {year})', pad=15)
    ax.set_xlabel('Total Visits')
    ax.set_ylabel('Provider')
    for i, v in enumerate(top_prov_visits['No. of Visit']):
        ax.text(v + 0.5, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_prov_mc = mc_df_month.sort_values('Total MC Given', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_mc, x='Total MC Given', y='Provider', hue='Provider', palette='Greens_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Total MC Given ({month_name} {year})', pad=15)
    ax.set_xlabel('Total MC (Days)')
    ax.set_ylabel('Provider')
    for i, v in enumerate(top_prov_mc['Total MC Given']):
        ax.text(v + 0.5, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(fig)
    plt.close(fig)

    if show_mc_pct:
        fig = plt.figure(figsize=(18, 9))
        top_visits_provs = set(mc_df_month.sort_values('No. of Visit', ascending=False).head(10)['Provider'])
        top_mc_provs = set(mc_df_month.sort_values('Total MC Given', ascending=False).head(10)['Provider'])
        top_provs = top_visits_provs.union(top_mc_provs)
        top_prov_mc_pct = mc_df_month[mc_df_month['Provider'].isin(top_provs)].sort_values(
            '% MC Given', ascending=(mc_sort_order == "asc")).head(20)
        ax = sns.barplot(data=top_prov_mc_pct, x='Provider', y='% MC Given', hue='Provider', 
                         palette='Purples_r', legend=False, edgecolor='black', linewidth=0.5)
        ax.set_title(f'Top 20 Providers by % MC Given ({month_name} {year}) - Sorted {"Ascending" if mc_sort_order == "asc" else "Descending"}', pad=15)
        ax.set_ylabel('% MC Given', fontsize=14)
        ax.set_xlabel('Provider', fontsize=14)
        plt.xticks(rotation=45, ha='right', fontsize=11)
        for i, v in enumerate(top_prov_mc_pct['% MC Given']):
            ax.text(i, v + 1, f'{v:.1f}%', ha='center', fontsize=10, color='#333333')
        plt.tight_layout()
        provider_charts.append(fig)
    else:
        provider_charts.append(None)
    if show_mc_pct:
        plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_prov_claim = claim_df_month.sort_values('Total Claim', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_claim, x='Total Claim', y='Provider Name', hue='Provider Name', 
                     palette='Oranges_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Total Claim ({month_name} {year})', pad=15)
    ax.set_xlabel('Total Claim ($)')
    ax.set_ylabel('Provider')
    for i, v in enumerate(top_prov_claim['Total Claim']):
        ax.text(v + 0.5, i, f'{v:,.2f}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_prov_avg_claim = claim_df_month.sort_values('Avg Claim per Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_avg_claim, x='Provider Name', y='Avg Claim per Visit', hue='Provider Name', 
                     palette='Reds_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Avg Claim per Visit ({month_name} {year})', pad=15)
    ax.set_ylabel('Avg Claim per Visit ($)')
    ax.set_xlabel('Provider')
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_prov_avg_claim['Avg Claim per Visit']):
        ax.text(i, v + 0.5, f'{v:.2f}', ha='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(fig)
    plt.close(fig)

    # Employee Charts
    fig = plt.figure(figsize=(12, 6))
    top_emp_visits = patient_df_month.sort_values('Total Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_visits, x='Total Visit', y='Employee Name', hue='Employee Name', 
                     palette='Blues_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Total Visits ({month_name} {year})', pad=15)
    ax.set_xlabel('Total Visits')
    ax.set_ylabel('Employee')
    for i, v in enumerate(top_emp_visits['Total Visit']):
        ax.text(v + 0.2, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_emp_claim = patient_df_month.sort_values('Total Claim (Combined)', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_claim, x='Total Claim (Combined)', y='Employee Name', hue='Employee Name', 
                     palette='Oranges_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Total Claim ({month_name} {year})', pad=15)
    ax.set_xlabel('Total Claim ($)')
    ax.set_ylabel('Employee')
    for i, v in enumerate(top_emp_claim['Total Claim (Combined)']):
        ax.text(v + 1, i, f'{v:,.2f}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_emp_avg_claim = patient_df_month.sort_values('Avg Claim per Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_avg_claim, x='Employee Name', y='Avg Claim per Visit', hue='Employee Name', 
                     palette='Reds_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Avg Claim per Visit ({month_name} {year})', pad=15)
    ax.set_ylabel('Avg Claim per Visit ($)')
    ax.set_xlabel('Employee')
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_emp_avg_claim['Avg Claim per Visit']):
        ax.text(i, v + 0.5, f'{v:.2f}', ha='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_emp_mc = patient_df_month.sort_values('Total MC (Days)', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_mc, x='Total MC (Days)', y='Employee Name', hue='Employee Name', 
                     palette='Greens_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Total MC ({month_name} {year})', pad=15)
    ax.set_xlabel('Total MC (Days)')
    ax.set_ylabel('Employee')
    for i, v in enumerate(top_emp_mc['Total MC (Days)']):
        ax.text(v + 0.2, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_emp_avg_mc = patient_df_month.sort_values('Avg MC per Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_avg_mc, x='Employee Name', y='Avg MC per Visit', hue='Employee Name', 
                     palette='Purples_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Avg MC per Visit ({month_name} {year})', pad=15)
    ax.set_ylabel('Avg MC per Visit (Days)')
    ax.set_xlabel('Employee')
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_emp_avg_mc['Avg MC per Visit']):
        ax.text(i, v + 0.05, f'{v:.2f}', ha='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(10, 6))
    division_claims = patient_df_month.groupby('Division/Department')['Total Claim (Combined)'].sum()
    plt.pie(division_claims, labels=division_claims.index, autopct='%1.1f%%', colors=sns.color_palette('muted'), 
            startangle=90, textprops={'fontsize': 11, 'color': '#333333'}, wedgeprops={'edgecolor': 'black', 'linewidth': 0.5})
    plt.title(f'Claim Distribution by Division ({month_name} {year})', pad=15)
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    provider_images = [fig_to_image(fig) if fig is not None else None for fig in provider_charts]
    employee_images = [fig_to_image(fig) for fig in employee_charts]
    
    print(f"Generated {len(provider_charts)} provider charts and {len(employee_charts)} employee charts for {month_name} {year}")
    return provider_images, employee_images, provider_charts, employee_charts

def fig_to_image(fig):
    if fig is None:
        return None
    fig.canvas.draw()
    img_array = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
    img_array = img_array.reshape(fig.canvas.get_width_height()[::-1] + (4,))
    return img_array

def charts_to_pptx(provider_charts, employee_charts, year):
    try:
        prs = Presentation()
        slide_layout = prs.slide_layouts[5]  # Blank slide layout with title
        prs.slide_width = Inches(13.33)  # Widescreen slide size
        prs.slide_height = Inches(7.5)

        # Define chart titles
        provider_titles = [
            f"Top 10 Providers by Total Visits ({year})",
            f"Top 10 Providers by Total MC Given ({year})",
            f"Top 20 Providers by % MC Given ({year})",
            f"Top 10 Providers by Total Claim ({year})",
            f"Top 10 Providers by Avg Claim per Visit ({year})"
        ]
        
        employee_titles = [
            f"Top 10 Employees by Total Visits ({year})",
            f"Top 10 Employees by Total Claim ({year})",
            f"Top 10 Employees by Avg Claim per Visit ({year})",
            f"Top 10 Employees by Total MC ({year})",
            f"Top 10 Employees by Avg MC per Visit ({year})",
            f"Claim Distribution by Division ({year})"
        ]

        # Provider Slide (all charts on one slide)
        if provider_charts:
            slide = prs.slides.add_slide(slide_layout)
            slide.shapes.title.text = f"Provider Insights ({year})"
            num_charts = sum(1 for fig in provider_charts if fig is not None)
            if num_charts == 0:
                print("No valid provider charts to export")
            else:
                charts_per_row = 3
                rows = (num_charts + charts_per_row - 1) // charts_per_row
                chart_width = Inches(3.8)
                chart_height = Inches(2.5) if rows > 1 else Inches(5)
                top_start = Inches(1.5)
                valid_charts = [fig for fig in provider_charts if fig is not None]
                
                for idx, fig in enumerate(valid_charts):
                    row = idx // charts_per_row
                    col = idx % charts_per_row
                    left = Inches(0.5 + col * 4.5)
                    top = top_start + row * (chart_height + Inches(0.2))
                    img_stream = BytesIO()
                    fig.savefig(img_stream, format='png', bbox_inches='tight', dpi=100)
                    img_stream.seek(0)
                    if img_stream.getvalue():
                        slide.shapes.add_picture(img_stream, left, top, width=chart_width, height=chart_height)
                        print(f"Added chart {idx+1}: '{provider_titles[idx]}' to Provider slide")
                    else:
                        print(f"Failed to save chart {idx+1}: '{provider_titles[idx]}' - empty image stream")

        # Employee Slide (all charts on one slide)
        if employee_charts:
            slide = prs.slides.add_slide(slide_layout)
            slide.shapes.title.text = f"Employee Insights ({year})"
            num_charts = sum(1 for fig in employee_charts if fig is not None)
            if num_charts == 0:
                print("No valid employee charts to export")
            else:
                charts_per_row = 3
                rows = (num_charts + charts_per_row - 1) // charts_per_row
                chart_width = Inches(3.8)
                chart_height = Inches(2.5) if rows > 1 else Inches(5)
                top_start = Inches(1.5)
                valid_charts = [fig for fig in employee_charts if fig is not None]
                
                for idx, fig in enumerate(valid_charts):
                    row = idx // charts_per_row
                    col = idx % charts_per_row
                    left = Inches(0.5 + col * 4.5)
                    top = top_start + row * (chart_height + Inches(0.2))
                    img_stream = BytesIO()
                    fig.savefig(img_stream, format='png', bbox_inches='tight', dpi=100)
                    img_stream.seek(0)
                    if img_stream.getvalue():
                        slide.shapes.add_picture(img_stream, left, top, width=chart_width, height=chart_height)
                        print(f"Added chart {idx+1}: '{employee_titles[idx]}' to Employee slide")
                    else:
                        print(f"Failed to save chart {idx+1}: '{employee_titles[idx]}' - empty image stream")

        with tempfile.NamedTemporaryFile(delete=False, suffix='.pptx') as tmp_file:
            prs.save(tmp_file.name)
            tmp_file_path = tmp_file.name

        print(f"PPTX file saved to temporary path: {tmp_file_path}")
        return tmp_file_path

    except Exception as e:
        print(f"Error generating PPTX: {str(e)}")
        return None

# --- Gradio Interface ---
with gr.Blocks(title="Claims Analysis Dashboard", css="""
    body { background-color: #f5f6f5; }
    h1, h2 { color: #333333; font-family: Arial; }
""") as demo:
    gr.Markdown("# Claims Analysis Dashboard (2024 - Present)")
    
    with gr.Row():
        url_input = gr.Textbox(label="Website URL", placeholder="Enter URL here", lines=1)
        user_id_input = gr.Textbox(label="User ID", placeholder="Enter User ID", lines=1)
        password_input = gr.Textbox(label="Password", type="password", placeholder="Enter Password", lines=1)
    scrape_btn = gr.Button("Submit", variant="primary")
    
    with gr.Row():
        year_dropdown = gr.Dropdown(
            label="Select Year to View Data",
            choices=["2024"],
            value="2024",
            allow_custom_value=False,
            interactive=False
        )
        month_dropdown = gr.Dropdown(
            label="Select Month (Optional)",
            choices=["Yearly"] + [f"{year}-{month:02d}" for year in range(2024, 2026) for month in range(1, 13)],
            value="Yearly",
            allow_custom_value=False,
            interactive=False
        )
        show_mc_pct_checkbox = gr.Checkbox(label="Show % MC Given Chart", value=True)
        mc_sort_dropdown = gr.Dropdown(
            label="Sort % MC Given",
            choices=["desc", "asc"],
            value="desc",
            allow_custom_value=False
        )
    
    status_output = gr.Textbox(label="Status", lines=2, interactive=False)
    patient_state = gr.State()
    claim_state = gr.State()
    mc_state = gr.State()

    with gr.Tabs():
        with gr.TabItem("Provider Insights"):
            gr.Markdown("## Provider Insights Dashboard")
            with gr.Row():
                prov_chart1 = gr.Image(label="Total Visits by Providers", interactive=False)
                prov_chart2 = gr.Image(label="Total MC by Providers", interactive=False)
            with gr.Row():
                prov_chart3 = gr.Image(label="% MC Given by Providers", interactive=False, visible=True)
            with gr.Row():
                prov_chart4 = gr.Image(label="Total Claim by Providers", interactive=False)
                prov_chart5 = gr.Image(label="Average Claim per Visit by Providers", interactive=False)
            download_btn_prov = gr.Button("Download Provider Charts as PPTX")

        with gr.TabItem("Employee Insights"):
            gr.Markdown("## Employee Insights Dashboard")
            with gr.Row():
                emp_chart1 = gr.Image(label="Total Visits by Employees", interactive=False)
                emp_chart2 = gr.Image(label="Total Claim by Employees", interactive=False)
            with gr.Row():
                emp_chart3 = gr.Image(label="Average Claim per Visit by Employees", interactive=False)
                emp_chart4 = gr.Image(label="Total MC by Employees", interactive=False)
            with gr.Row():
                emp_chart5 = gr.Image(label="Average MC per Visit by Employees", interactive=False)
                emp_chart6 = gr.Image(label="Claim Distribution by Division", interactive=False)
            download_btn_emp = gr.Button("Download Employee Charts as PPTX")

    def scrape_and_store(url, user_id, password, show_mc_pct, mc_sort_order):
        patient_data_by_year, claim_data_by_year, mc_data_by_year, status = scrape_data(url, user_id, password)
        if patient_data_by_year is None or claim_data_by_year is None or mc_data_by_year is None:
            return (
                status, None, None, None, 
                gr.update(choices=["2024"], value="2024", interactive=False),
                gr.update(choices=["Yearly"], value="Yearly", interactive=False),
                None, None, None, None, None, None, None, None, None, None, None
            )

        available_years = sorted(set(patient_data_by_year.keys()) | set(claim_data_by_year.keys()) | set(mc_data_by_year.keys()))
        year_choices = [str(year) for year in available_years]
        default_year = year_choices[0] if year_choices else "2024"
        month_choices = ["Yearly"] + [f"{year}-{month:02d}" for year in available_years for month in range(1, 13)]

        provider_images, employee_images, provider_figs, employee_figs = generate_dashboard_charts(
            patient_data_by_year, claim_data_by_year, mc_data_by_year, default_year, show_mc_pct, mc_sort_order)
        return (
            status, patient_data_by_year, claim_data_by_year, mc_data_by_year,
            gr.update(choices=year_choices, value=default_year, interactive=True),
            gr.update(choices=month_choices, value="Yearly", interactive=True),
            provider_images[0], provider_images[1], provider_images[2], provider_images[3], provider_images[4],
            employee_images[0], employee_images[1], employee_images[2], employee_images[3], employee_images[4], employee_images[5]
        )

    def update_dashboard(year, month, patient_data_by_year, claim_data_by_year, mc_data_by_year, show_mc_pct, mc_sort_order):
        if not patient_data_by_year or not claim_data_by_year or not mc_data_by_year:
            return [None] * 11
        if month == "Yearly":
            provider_images, employee_images, _, _ = generate_dashboard_charts(
                patient_data_by_year, claim_data_by_year, mc_data_by_year, year, show_mc_pct, mc_sort_order)
        else:
            provider_images, employee_images, _, _ = generate_monthly_dashboard_charts(
                patient_data_by_year, claim_data_by_year, mc_data_by_year, year, month, show_mc_pct, mc_sort_order)
        return (
            provider_images[0], provider_images[1], provider_images[2], provider_images[3], provider_images[4],
            employee_images[0], employee_images[1], employee_images[2], employee_images[3], employee_images[4], employee_images[5]
        )

    def download_provider_pptx(year, month, patient_data_by_year, claim_data_by_year, mc_data_by_year, show_mc_pct, mc_sort_order):
        if month == "Yearly":
            _, _, provider_figs, _ = generate_dashboard_charts(
                patient_data_by_year, claim_data_by_year, mc_data_by_year, year, show_mc_pct, mc_sort_order)
            pptx_path = charts_to_pptx(provider_figs, [], year)
            label = f"Provider_Charts_{year}.pptx"
        else:
            _, _, provider_figs, _ = generate_monthly_dashboard_charts(
                patient_data_by_year, claim_data_by_year, mc_data_by_year, year, month, show_mc_pct, mc_sort_order)
            month_name = datetime.strptime(month.split('-')[1], '%m').strftime('%B')
            pptx_path = charts_to_pptx(provider_figs, [], f"{month_name} {year}")
            label = f"Provider_Charts_{month_name}_{year}.pptx"
        if pptx_path:
            return gr.File(value=pptx_path, label=label)
        else:
            print("Failed to generate provider PPTX")
            return None

    def download_employee_pptx(year, month, patient_data_by_year, claim_data_by_year, mc_data_by_year, show_mc_pct, mc_sort_order):
        if month == "Yearly":
            _, _, _, employee_figs = generate_dashboard_charts(
                patient_data_by_year, claim_data_by_year, mc_data_by_year, year, show_mc_pct, mc_sort_order)
            pptx_path = charts_to_pptx([], employee_figs, year)
            label = f"Employee_Charts_{year}.pptx"
        else:
            _, _, _, employee_figs = generate_monthly_dashboard_charts(
                patient_data_by_year, claim_data_by_year, mc_data_by_year, year, month, show_mc_pct, mc_sort_order)
            month_name = datetime.strptime(month.split('-')[1], '%m').strftime('%B')
            pptx_path = charts_to_pptx([], employee_figs, f"{month_name} {year}")
            label = f"Employee_Charts_{month_name}_{year}.pptx"
        if pptx_path:
            return gr.File(value=pptx_path, label=label)
        else:
            print("Failed to generate employee PPTX")
            return None

    scrape_btn.click(
        fn=scrape_and_store,
        inputs=[url_input, user_id_input, password_input, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=[
            status_output, patient_state, claim_state, mc_state, year_dropdown, month_dropdown,
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6
        ]
    )

    year_dropdown.change(
        fn=update_dashboard,
        inputs=[year_dropdown, month_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=[
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6
        ]
    )

    month_dropdown.change(
        fn=update_dashboard,
        inputs=[year_dropdown, month_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=[
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6
        ]
    )

    show_mc_pct_checkbox.change(
        fn=update_dashboard,
        inputs=[year_dropdown, month_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=[
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6
        ]
    )

    mc_sort_dropdown.change(
        fn=update_dashboard,
        inputs=[year_dropdown, month_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=[
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6
        ]
    )

    download_btn_prov.click(
        fn=download_provider_pptx,
        inputs=[year_dropdown, month_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=gr.File()
    )

    download_btn_emp.click(
        fn=download_employee_pptx,
        inputs=[year_dropdown, month_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=gr.File()
    )

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7867
* Running on public URL: https://330b0137aaeb1690d4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Generated 5 provider charts and 6 employee charts for year 2024
Generated 5 provider charts and 6 employee charts for January 2024


In [8]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from datetime import datetime, timedelta
import time
import numpy as np
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException
from io import BytesIO
from pptx import Presentation
from pptx.util import Inches
import tempfile

# --- Scraping Functions ---
def wait_for_element(driver, locator):
    return WebDriverWait(driver, 10).until(EC.element_to_be_clickable(locator))

def select_date_month_day(driver, date_str, date_input_id):
    date_to_select = datetime.strptime(date_str, '%Y-%m-%d')
    date_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, date_input_id)))
    date_input.click()
    month_select = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-datepicker-month')))
    month_option = month_select.find_element(By.XPATH, f"//option[@value='{date_to_select.month - 1}']")
    month_option.click()
    day = date_to_select.day
    day_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//td[@data-handler='selectDay']/a[text()='{day}']")))
    day_element.click()
    time.sleep(5)

def select_date(driver, date_str, date_input_id):
    date_to_select = datetime.strptime(date_str, '%Y-%m-%d')
    date_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, date_input_id)))
    date_input.click()
    time.sleep(2)
    month_select = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-datepicker-month')))
    month_option = month_select.find_element(By.XPATH, f"//option[@value='{date_to_select.month - 1}']")
    time.sleep(2)
    month_option.click()
    year_select = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-datepicker-year')))
    year_option = year_select.find_element(By.XPATH, f"//option[@value='{date_to_select.year}']")
    year_option.click()
    day = date_to_select.day
    day_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//td[@data-handler='selectDay']/a[text()='{day}']")))
    day_element.click()
    time.sleep(5)

def extract_grid_data_clm_summary(driver):
    data = []
    try:
        total_pages_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "sp_1_pjqgridClmSummbyProv")))
        total_pages = int(total_pages_element.text.strip())
    except:
        return data
    for current_page in range(1, total_pages + 1):
        time.sleep(3)
        driver.execute_script("window.scrollTo(0, 0);")
        grid = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "jqgridClmSummbyProv")))
        rows = WebDriverWait(grid, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "tr.jqgrow")))
        for row in rows:
            try:
                provider_name = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_ProvName']").text
                visits = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_NoOfVisit']").text
                claim = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_ClmAmt']").text
                try:
                    total_mc = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridClmSummbyProv_STR_TotalMC']").text
                except:
                    total_mc = '0'
                data.append({'Provider Name': provider_name, 'No of Visits': visits, 'Total Claim': claim, 'Total MC (Days)': total_mc})
            except Exception as e:
                print(f"Error extracting row: {e}")
                continue
        if current_page < total_pages:
            try:
                next_button_div = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn.btn-sm.btn-default span.fa.fa-forward")))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button_div)
                next_button_div.click()
                WebDriverWait(driver, 10).until(EC.staleness_of(rows[0]))
            except:
                break
    return data

def extract_grid_data_patient_analysis(driver):
    all_data = []
    while True:
        driver.execute_script("window.scrollTo(0, 0);")
        grid = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "jqgridCorpMcAnalysis")))
        rows = grid.find_elements(By.CSS_SELECTOR, "tr.jqgrow")
        for row in rows:
            try:
                employee_name = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_MEM_NAME']").text
                employee_no = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_MEM_EMPID']").text
                division = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_MEM_EMPDIVISION']").text
                total_visit = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalVisit']").text
                total_mc = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalMC']").text
                total_claim_own = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalClaim_Own']").text
                total_claim_dep = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgridCorpMcAnalysis_totalClaim_Dep']").text
                all_data.append({
                    'Employee Name': employee_name, 'Employee No': employee_no, 'Division/Department': division,
                    'Total Visit': total_visit, 'Total MC (Days)': total_mc, 'Total Claim (Own)': total_claim_own,
                    'Total Claim (Dep)': total_claim_dep
                })
            except:
                continue
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            next_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn.btn-sm.btn-default span.fa.fa-forward")))
            parent_div = next_button.find_element(By.XPATH, "./parent::div")
            if "disabled" in parent_div.get_attribute("class"):
                break
            driver.execute_script("arguments[0].scrollIntoView(true);", parent_div)
            parent_div.click()
            WebDriverWait(driver, 10).until(EC.staleness_of(rows[0]))
        except:
            break
    return all_data

def extract_grid_data_mc(driver):
    data = []
    try:
        total_pages_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "sp_1_jqgrid")))
        total_pages = int(total_pages_element.text.strip())
    except:
        total_pages = 1
    for current_page in range(1, total_pages + 1):
        time.sleep(3)
        driver.execute_script("window.scrollTo(0, 0);")
        grid = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "jqgrid")))
        rows = WebDriverWait(grid, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "tr.jqgrow")))
        for row in rows:
            try:
                provider_name = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgrid_STR_ProvName']").text.strip()
                total_mc_given = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgrid_STR_MC_Given_Count']").text.strip()
                total_visits = row.find_element(By.CSS_SELECTOR, "td[aria-describedby='jqgrid_STR_VISITCount']").text.strip()
                data.append({'Provider': provider_name, 'Total MC Given': total_mc_given, 'No. of Visit': total_visits})
            except:
                continue
        if current_page < total_pages:
            try:
                next_button_div = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn.btn-sm.btn-default span.fa.fa-forward")))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button_div)
                next_button_div.click()
                WebDriverWait(driver, 10).until(EC.staleness_of(rows[0]))
            except:
                break
    return data

def scrape_data(url, user_id, password):
    edge_options = Options()
    driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()), options=edge_options)
    start_year = 2024
    current_date = datetime.now().strftime('%Y-%m-%d')
    patient_data_by_year = {}
    claim_data_by_year = {}
    mc_data_by_year = {}
    try:
        driver.get(url)
        image = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//img[@src='/ClaimEXMVR/Servlet_LoadImage?SFC=loadImage&imageName=icorporate.png']")))
        image.click()
        user_id_field = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.NAME, "txtloginid")))
        user_id_field.send_keys(user_id)
        password_field = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "inputpss")))
        password_field.send_keys(password)
        sign_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn.btn-primary[type='submit']")))
        sign_button.click()
        continue_button = wait_for_element(driver, (By.XPATH, "//button[text()='Continue']"))
        continue_button.click()

        for year in range(start_year, 2025 + 1):
            if year == 2024:
                start_date = "2024-01-01"
                end_date = "2024-12-31"
            else:
                start_date = "2025-01-01"
                end_date = current_date
            productivity_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[span[text()='Productivity Reports']]")))
            productivity_link.click()
            patient_analysis_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Patient_Analysis_Report'][span[text()=' Patient Analysis Report ']]")))
            patient_analysis_link.click()
            select_date(driver, start_date, "txtStartDate")
            select_date(driver, end_date, "txtEndDate")
            search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
            search_button.click()
            time.sleep(5)
            dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
            driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
            select = Select(dropdown)
            select.select_by_value("100")
            time.sleep(10)
            patient_data = extract_grid_data_patient_analysis(driver)
            patient_df = pd.DataFrame(patient_data)
            numeric_cols_patient = ['Total Visit', 'Total MC (Days)', 'Total Claim (Own)', 'Total Claim (Dep)']
            for col in numeric_cols_patient:
                patient_df[col] = pd.to_numeric(patient_df[col], errors='coerce')
            patient_df['Total Claim (Combined)'] = patient_df['Total Claim (Own)'] + patient_df['Total Claim (Dep)']
            patient_df['Avg Claim per Visit'] = patient_df['Total Claim (Combined)'] / patient_df['Total Visit']
            patient_df['Avg MC per Visit'] = patient_df['Total MC (Days)'] / patient_df['Total Visit']
            patient_df['Avg Claim per MC'] = patient_df['Total Claim (Combined)'] / patient_df['Total MC (Days)']
            patient_data_by_year[year] = patient_df

        for year in range(start_year, 2025 + 1):
            if year == 2024:
                start_date = "2024-01-01"
                end_date = "2024-12-31"
            else:
                start_date = "2025-01-01"
                end_date = current_date
            productivity_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[span[text()='Productivity Reports']]")))
            productivity_link.click()
            mc_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/MC_HealthCare_By_Provider'][span[text()=' MC by Provider ']]")))
            mc_link.click()
            time.sleep(2)
            select_date(driver, start_date, "txtStartDate")
            select_date(driver, end_date, "txtEndDate")
            search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
            search_button.click()
            time.sleep(10)
            dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
            driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
            select = Select(dropdown)
            select.select_by_value("100")
            time.sleep(10)
            mc_data = extract_grid_data_mc(driver)
            mc_df = pd.DataFrame(mc_data)
            numeric_cols_mc = ['Total MC Given', 'No. of Visit']
            for col in numeric_cols_mc:
                mc_df[col] = pd.to_numeric(mc_df[col], errors='coerce')
            mc_df['% MC Given'] = (mc_df['Total MC Given'] / mc_df['No. of Visit']) * 100
            mc_data_by_year[year] = mc_df

        start_date = "2024-01-01"
        end_date = "2025-01-01"
        reg_claims_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[.//span[contains(text(), 'Registration') and contains(text(), 'Claims')]]")))
        reg_claims_link.click()
        providers_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Claim_Summary_by_Provider_Analysis'][span[text()=' Claim Summary by Providers ']]")))
        providers_link.click()
        time.sleep(5)
        select_date_month_day(driver, start_date, "txtFromDate")
        select_date_month_day(driver, end_date, "txtToDate")
        search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
        driver.execute_script("arguments[0].click();", search_button)
        time.sleep(10)
        dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
        driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
        select = Select(dropdown)
        select.select_by_value("100")
        time.sleep(5)
        claim_data_2024 = extract_grid_data_clm_summary(driver)
        claim_df_2024 = pd.DataFrame(claim_data_2024)
        numeric_cols_claim = ['No of Visits', 'Total Claim', 'Total MC (Days)']
        for col in numeric_cols_claim:
            claim_df_2024[col] = pd.to_numeric(claim_df_2024[col], errors='coerce')
        claim_df_2024['Avg Claim per Visit'] = claim_df_2024['Total Claim'] / claim_df_2024['No of Visits']
        claim_data_by_year[2024] = claim_df_2024

        start_date = "2024-12-31"
        end_date = current_date
        reg_claims_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[.//span[contains(text(), 'Registration') and contains(text(), 'Claims')]]")))
        reg_claims_link.click()
        providers_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Claim_Summary_by_Provider_Analysis'][span[text()=' Claim Summary by Providers ']]")))
        providers_link.click()
        time.sleep(5)
        select_date_month_day(driver, start_date, "txtFromDate")
        select_date_month_day(driver, end_date, "txtToDate")
        search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
        driver.execute_script("arguments[0].click();", search_button)
        time.sleep(10)
        dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
        driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
        select = Select(dropdown)
        select.select_by_value("100")
        time.sleep(5)
        claim_data_recent = extract_grid_data_clm_summary(driver)
        claim_df_recent = pd.DataFrame(claim_data_recent)
        for col in numeric_cols_claim:
            claim_df_recent[col] = pd.to_numeric(claim_df_recent[col], errors='coerce')
        claim_df_recent['Avg Claim per Visit'] = claim_df_recent['Total Claim'] / claim_df_recent['No of Visits']
        claim_data_by_year[2025] = claim_df_recent

        return patient_data_by_year, claim_data_by_year, mc_data_by_year, "Data scraped successfully!"
    except Exception as e:
        return None, None, None, f"Error: {str(e)}"
    finally:
        driver.quit()

def scrape_monthly_data(url, user_id, password):
    edge_options = Options()
    driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()), options=edge_options)
    start_year = 2024
    current_date = datetime.strptime(datetime.now().strftime('%Y-%m-%d'), '%Y-%m-%d')
    patient_data_by_month = {}
    claim_data_by_month = {}
    mc_data_by_month = {}
    try:
        driver.get(url)
        image = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//img[@src='/ClaimEXMVR/Servlet_LoadImage?SFC=loadImage&imageName=icorporate.png']")))
        image.click()
        user_id_field = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.NAME, "txtloginid")))
        user_id_field.send_keys(user_id)
        password_field = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "inputpss")))
        password_field.send_keys(password)
        sign_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn.btn-primary[type='submit']")))
        sign_button.click()
        continue_button = wait_for_element(driver, (By.XPATH, "//button[text()='Continue']"))
        continue_button.click()

        start_date = datetime.strptime("2024-01-01", '%Y-%m-%d')
        while start_date <= current_date:
            end_date = (start_date.replace(day=1, month=start_date.month % 12 + 1, year=start_date.year + start_date.month // 12) - timedelta(days=1)) if start_date.month < 12 else start_date.replace(day=31)
            if end_date > current_date:
                end_date = current_date
            month_key = start_date.strftime('%Y-%m')

            # Patient Analysis
            productivity_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[span[text()='Productivity Reports']]")))
            productivity_link.click()
            patient_analysis_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Patient_Analysis_Report'][span[text()=' Patient Analysis Report ']]")))
            patient_analysis_link.click()
            select_date(driver, start_date.strftime('%Y-%m-%d'), "txtStartDate")
            select_date(driver, end_date.strftime('%Y-%m-%d'), "txtEndDate")
            search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
            search_button.click()
            time.sleep(5)
            dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
            driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
            select = Select(dropdown)
            select.select_by_value("100")
            time.sleep(10)
            patient_data = extract_grid_data_patient_analysis(driver)
            patient_df = pd.DataFrame(patient_data)
            numeric_cols_patient = ['Total Visit', 'Total MC (Days)', 'Total Claim (Own)', 'Total Claim (Dep)']
            for col in numeric_cols_patient:
                patient_df[col] = pd.to_numeric(patient_df[col], errors='coerce')
            patient_df['Total Claim (Combined)'] = patient_df['Total Claim (Own)'] + patient_df['Total Claim (Dep)']
            patient_df['Avg Claim per Visit'] = patient_df['Total Claim (Combined)'] / patient_df['Total Visit']
            patient_df['Avg MC per Visit'] = patient_df['Total MC (Days)'] / patient_df['Total Visit']
            patient_df['Avg Claim per MC'] = patient_df['Total Claim (Combined)'] / patient_df['Total MC (Days)']
            patient_data_by_month[month_key] = patient_df

            # MC by Provider
            productivity_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[span[text()='Productivity Reports']]")))
            productivity_link.click()
            mc_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/MC_HealthCare_By_Provider'][span[text()=' MC by Provider ']]")))
            mc_link.click()
            time.sleep(2)
            select_date(driver, start_date.strftime('%Y-%m-%d'), "txtStartDate")
            select_date(driver, end_date.strftime('%Y-%m-%d'), "txtEndDate")
            search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
            search_button.click()
            time.sleep(10)
            dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
            driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
            select = Select(dropdown)
            select.select_by_value("100")
            time.sleep(10)
            mc_data = extract_grid_data_mc(driver)
            mc_df = pd.DataFrame(mc_data)
            numeric_cols_mc = ['Total MC Given', 'No. of Visit']
            for col in numeric_cols_mc:
                mc_df[col] = pd.to_numeric(mc_df[col], errors='coerce')
            mc_df['% MC Given'] = (mc_df['Total MC Given'] / mc_df['No. of Visit']) * 100
            mc_data_by_month[month_key] = mc_df

            # Claim Summary by Providers
            reg_claims_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[.//span[contains(text(), 'Registration') and contains(text(), 'Claims')]]")))
            reg_claims_link.click()
            providers_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='#/Claim_Summary_by_Provider_Analysis'][span[text()=' Claim Summary by Providers ']]")))
            providers_link.click()
            time.sleep(5)
            select_date_month_day(driver, start_date.strftime('%Y-%m-%d'), "txtFromDate")
            select_date_month_day(driver, end_date.strftime('%Y-%m-%d'), "txtToDate")
            search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
            driver.execute_script("arguments[0].click();", search_button)
            time.sleep(10)
            dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.ui-pg-selbox")))
            driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
            select = Select(dropdown)
            select.select_by_value("100")
            time.sleep(5)
            claim_data = extract_grid_data_clm_summary(driver)
            claim_df = pd.DataFrame(claim_data)
            numeric_cols_claim = ['No of Visits', 'Total Claim', 'Total MC (Days)']
            for col in numeric_cols_claim:
                claim_df[col] = pd.to_numeric(claim_df[col], errors='coerce')
            claim_df['Avg Claim per Visit'] = claim_df['Total Claim'] / claim_df['No of Visits']
            claim_data_by_month[month_key] = claim_df

            start_date = end_date + timedelta(days=1)

        return patient_data_by_month, claim_data_by_month, mc_data_by_month, "Monthly data scraped successfully!"
    except Exception as e:
        return None, None, None, f"Error: {str(e)}"
    finally:
        driver.quit()

# --- Plotting Functions ---
def generate_dashboard_charts(patient_data, claim_data, mc_data, period, show_mc_pct=True, mc_sort_order="desc"):
    if not patient_data or not claim_data or not mc_data:
        print("No data available to generate charts.")
        return None, None, None, None
    
    patient_df = patient_data.get(period, pd.DataFrame())
    claim_df = claim_data.get(period, pd.DataFrame())
    mc_df = mc_data.get(period, pd.DataFrame())
    
    if patient_df.empty or claim_df.empty or mc_df.empty:
        print(f"Data for period {period} is empty: Patient: {patient_df.empty}, Claim: {claim_df.empty}, MC: {mc_df.empty}")
        return None, None, None, None
    
    sns.set(style="whitegrid", palette="muted")
    plt.rcParams.update({
        'font.family': 'Arial', 'font.size': 12, 'axes.titlesize': 16, 
        'axes.labelsize': 14, 'xtick.labelsize': 11, 'ytick.labelsize': 11,
        'axes.titleweight': 'bold', 'axes.linewidth': 1.5, 'grid.linestyle': ':', 
        'grid.alpha': 0.5, 'figure.facecolor': '#f5f6f5', 'axes.facecolor': '#ffffff',
        'axes.edgecolor': '#333333', 'axes.labelcolor': '#333333', 'text.color': '#333333'
    })
    
    provider_charts = []
    employee_charts = []

    # Provider Charts
    fig = plt.figure(figsize=(12, 6))
    top_prov_visits = mc_df.sort_values('No. of Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_visits, x='No. of Visit', y='Provider', hue='Provider', palette='Blues_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Total Visits ({period})', pad=15)
    ax.set_xlabel('Total Visits')
    ax.set_ylabel('Provider')
    for i, v in enumerate(top_prov_visits['No. of Visit']):
        ax.text(v + 0.5, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_prov_mc = mc_df.sort_values('Total MC Given', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_mc, x='Total MC Given', y='Provider', hue='Provider', palette='Greens_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Total MC Given ({period})', pad=15)
    ax.set_xlabel('Total MC (Days)')
    ax.set_ylabel('Provider')
    for i, v in enumerate(top_prov_mc['Total MC Given']):
        ax.text(v + 0.5, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(fig)
    plt.close(fig)

    if show_mc_pct:
        fig = plt.figure(figsize=(18, 9))
        top_visits_provs = set(mc_df.sort_values('No. of Visit', ascending=False).head(10)['Provider'])
        top_mc_provs = set(mc_df.sort_values('Total MC Given', ascending=False).head(10)['Provider'])
        top_provs = top_visits_provs.union(top_mc_provs)
        top_prov_mc_pct = mc_df[mc_df['Provider'].isin(top_provs)].sort_values(
            '% MC Given', ascending=(mc_sort_order == "asc")).head(20)
        ax = sns.barplot(data=top_prov_mc_pct, x='Provider', y='% MC Given', hue='Provider', 
                         palette='Purples_r', legend=False, edgecolor='black', linewidth=0.5)
        ax.set_title(f'Top 20 Providers by % MC Given ({period}) - Sorted {"Ascending" if mc_sort_order == "asc" else "Descending"}', pad=15)
        ax.set_ylabel('% MC Given', fontsize=14)
        ax.set_xlabel('Provider', fontsize=14)
        plt.xticks(rotation=45, ha='right', fontsize=11)
        for i, v in enumerate(top_prov_mc_pct['% MC Given']):
            ax.text(i, v + 1, f'{v:.1f}%', ha='center', fontsize=10, color='#333333')
        plt.tight_layout()
        provider_charts.append(fig)
    else:
        provider_charts.append(None)
    if show_mc_pct:
        plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_prov_claim = claim_df.sort_values('Total Claim', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_claim, x='Total Claim', y='Provider Name', hue='Provider Name', 
                     palette='Oranges_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Total Claim ({period})', pad=15)
    ax.set_xlabel('Total Claim ($)')
    ax.set_ylabel('Provider')
    for i, v in enumerate(top_prov_claim['Total Claim']):
        ax.text(v + 0.5, i, f'{v:,.2f}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_prov_avg_claim = claim_df.sort_values('Avg Claim per Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_prov_avg_claim, x='Provider Name', y='Avg Claim per Visit', hue='Provider Name', 
                     palette='Reds_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Providers by Avg Claim per Visit ({period})', pad=15)
    ax.set_ylabel('Avg Claim per Visit ($)')
    ax.set_xlabel('Provider')
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_prov_avg_claim['Avg Claim per Visit']):
        ax.text(i, v + 0.5, f'{v:.2f}', ha='center', fontsize=10, color='#333333')
    plt.tight_layout()
    provider_charts.append(fig)
    plt.close(fig)

    # Employee Charts
    fig = plt.figure(figsize=(12, 6))
    top_emp_visits = patient_df.sort_values('Total Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_visits, x='Total Visit', y='Employee Name', hue='Employee Name', 
                     palette='Blues_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Total Visits ({period})', pad=15)
    ax.set_xlabel('Total Visits')
    ax.set_ylabel('Employee')
    for i, v in enumerate(top_emp_visits['Total Visit']):
        ax.text(v + 0.2, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_emp_claim = patient_df.sort_values('Total Claim (Combined)', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_claim, x='Total Claim (Combined)', y='Employee Name', hue='Employee Name', 
                     palette='Oranges_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Total Claim ({period})', pad=15)
    ax.set_xlabel('Total Claim ($)')
    ax.set_ylabel('Employee')
    for i, v in enumerate(top_emp_claim['Total Claim (Combined)']):
        ax.text(v + 1, i, f'{v:,.2f}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_emp_avg_claim = patient_df.sort_values('Avg Claim per Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_avg_claim, x='Employee Name', y='Avg Claim per Visit', hue='Employee Name', 
                     palette='Reds_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Avg Claim per Visit ({period})', pad=15)
    ax.set_ylabel('Avg Claim per Visit ($)')
    ax.set_xlabel('Employee')
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_emp_avg_claim['Avg Claim per Visit']):
        ax.text(i, v + 0.5, f'{v:.2f}', ha='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_emp_mc = patient_df.sort_values('Total MC (Days)', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_mc, x='Total MC (Days)', y='Employee Name', hue='Employee Name', 
                     palette='Greens_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Total MC ({period})', pad=15)
    ax.set_xlabel('Total MC (Days)')
    ax.set_ylabel('Employee')
    for i, v in enumerate(top_emp_mc['Total MC (Days)']):
        ax.text(v + 0.2, i, f'{int(v)}', va='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(12, 6))
    top_emp_avg_mc = patient_df.sort_values('Avg MC per Visit', ascending=False).head(10)
    ax = sns.barplot(data=top_emp_avg_mc, x='Employee Name', y='Avg MC per Visit', hue='Employee Name', 
                     palette='Purples_r', legend=False, edgecolor='black', linewidth=0.5)
    ax.set_title(f'Top 10 Employees by Avg MC per Visit ({period})', pad=15)
    ax.set_ylabel('Avg MC per Visit (Days)')
    ax.set_xlabel('Employee')
    plt.xticks(rotation=45, ha='right')
    for i, v in enumerate(top_emp_avg_mc['Avg MC per Visit']):
        ax.text(i, v + 0.05, f'{v:.2f}', ha='center', fontsize=10, color='#333333')
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    fig = plt.figure(figsize=(10, 6))
    division_claims = patient_df.groupby('Division/Department')['Total Claim (Combined)'].sum()
    plt.pie(division_claims, labels=division_claims.index, autopct='%1.1f%%', colors=sns.color_palette('muted'), 
            startangle=90, textprops={'fontsize': 11, 'color': '#333333'}, wedgeprops={'edgecolor': 'black', 'linewidth': 0.5})
    plt.title(f'Claim Distribution by Division ({period})', pad=15)
    plt.tight_layout()
    employee_charts.append(fig)
    plt.close(fig)

    provider_images = [fig_to_image(fig) if fig is not None else None for fig in provider_charts]
    employee_images = [fig_to_image(fig) for fig in employee_charts]
    
    print(f"Generated {len(provider_charts)} provider charts and {len(employee_charts)} employee charts for period {period}")
    return provider_images, employee_images, provider_charts, employee_charts

def fig_to_image(fig):
    if fig is None:
        return None
    fig.canvas.draw()
    img_array = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
    img_array = img_array.reshape(fig.canvas.get_width_height()[::-1] + (4,))
    return img_array

def charts_to_pptx(provider_charts, employee_charts, period):
    try:
        prs = Presentation()
        slide_layout = prs.slide_layouts[5]  # Blank slide layout with title
        prs.slide_width = Inches(13.33)  # Widescreen slide size
        prs.slide_height = Inches(7.5)

        provider_titles = [
            f"Top 10 Providers by Total Visits ({period})",
            f"Top 10 Providers by Total MC Given ({period})",
            f"Top 20 Providers by % MC Given ({period})",
            f"Top 10 Providers by Total Claim ({period})",
            f"Top 10 Providers by Avg Claim per Visit ({period})"
        ]
        
        employee_titles = [
            f"Top 10 Employees by Total Visits ({period})",
            f"Top 10 Employees by Total Claim ({period})",
            f"Top 10 Employees by Avg Claim per Visit ({period})",
            f"Top 10 Employees by Total MC ({period})",
            f"Top 10 Employees by Avg MC per Visit ({period})",
            f"Claim Distribution by Division ({period})"
        ]

        if provider_charts:
            slide = prs.slides.add_slide(slide_layout)
            slide.shapes.title.text = f"Provider Insights ({period})"
            num_charts = sum(1 for fig in provider_charts if fig is not None)
            if num_charts == 0:
                print("No valid provider charts to export")
            else:
                charts_per_row = 3
                rows = (num_charts + charts_per_row - 1) // charts_per_row
                chart_width = Inches(3.8)
                chart_height = Inches(2.5) if rows > 1 else Inches(5)
                top_start = Inches(1.5)
                valid_charts = [fig for fig in provider_charts if fig is not None]
                
                for idx, fig in enumerate(valid_charts):
                    row = idx // charts_per_row
                    col = idx % charts_per_row
                    left = Inches(0.5 + col * 4.5)
                    top = top_start + row * (chart_height + Inches(0.2))
                    img_stream = BytesIO()
                    fig.savefig(img_stream, format='png', bbox_inches='tight', dpi=100)
                    img_stream.seek(0)
                    if img_stream.getvalue():
                        slide.shapes.add_picture(img_stream, left, top, width=chart_width, height=chart_height)
                        print(f"Added chart {idx+1}: '{provider_titles[idx]}' to Provider slide")
                    else:
                        print(f"Failed to save chart {idx+1}: '{provider_titles[idx]}' - empty image stream")

        if employee_charts:
            slide = prs.slides.add_slide(slide_layout)
            slide.shapes.title.text = f"Employee Insights ({period})"
            num_charts = sum(1 for fig in employee_charts if fig is not None)
            if num_charts == 0:
                print("No valid employee charts to export")
            else:
                charts_per_row = 3
                rows = (num_charts + charts_per_row - 1) // charts_per_row
                chart_width = Inches(3.8)
                chart_height = Inches(2.5) if rows > 1 else Inches(5)
                top_start = Inches(1.5)
                valid_charts = [fig for fig in employee_charts if fig is not None]
                
                for idx, fig in enumerate(valid_charts):
                    row = idx // charts_per_row
                    col = idx % charts_per_row
                    left = Inches(0.5 + col * 4.5)
                    top = top_start + row * (chart_height + Inches(0.2))
                    img_stream = BytesIO()
                    fig.savefig(img_stream, format='png', bbox_inches='tight', dpi=100)
                    img_stream.seek(0)
                    if img_stream.getvalue():
                        slide.shapes.add_picture(img_stream, left, top, width=chart_width, height=chart_height)
                        print(f"Added chart {idx+1}: '{employee_titles[idx]}' to Employee slide")
                    else:
                        print(f"Failed to save chart {idx+1}: '{employee_titles[idx]}' - empty image stream")

        with tempfile.NamedTemporaryFile(delete=False, suffix='.pptx') as tmp_file:
            prs.save(tmp_file.name)
            tmp_file_path = tmp_file.name

        print(f"PPTX file saved to temporary path: {tmp_file_path}")
        return tmp_file_path

    except Exception as e:
        print(f"Error generating PPTX: {str(e)}")
        return None

# --- Gradio Interface ---
with gr.Blocks(title="Claims Analysis Dashboard", css="""
    body { background-color: #f5f6f5; }
    h1, h2 { color: #333333; font-family: Arial; }
""") as demo:
    gr.Markdown("# Claims Analysis Dashboard (2024 - Present)")
    
    with gr.Row():
        url_input = gr.Textbox(label="Website URL", placeholder="Enter URL here", lines=1)
        user_id_input = gr.Textbox(label="User ID", placeholder="Enter User ID", lines=1)
        password_input = gr.Textbox(label="Password", type="password", placeholder="Enter Password", lines=1)
    
    with gr.Row():
        scrape_yearly_btn = gr.Button("Scrape Yearly Data", variant="primary")
        scrape_monthly_btn = gr.Button("Scrape Monthly Data", variant="secondary")
    
    with gr.Row():
        year_dropdown = gr.Dropdown(
            label="Select Year to View Data",
            choices=["2024"],
            value="2024",
            allow_custom_value=False,
            interactive=False
        )
        month_dropdown = gr.Dropdown(
            label="Select Month (Optional)",
            choices=["Yearly"] + [f"{year}-{month:02d}" for year in range(2024, 2026) for month in range(1, 13)],
            value="Yearly",
            allow_custom_value=False,
            interactive=False
        )
        show_mc_pct_checkbox = gr.Checkbox(label="Show % MC Given Chart", value=True)
        mc_sort_dropdown = gr.Dropdown(
            label="Sort % MC Given",
            choices=["desc", "asc"],
            value="desc",
            allow_custom_value=False
        )
    
    status_output = gr.Textbox(label="Status", lines=2, interactive=False)
    patient_state = gr.State()
    claim_state = gr.State()
    mc_state = gr.State()

    with gr.Tabs():
        with gr.TabItem("Provider Insights"):
            gr.Markdown("## Provider Insights Dashboard")
            with gr.Row():
                prov_chart1 = gr.Image(label="Total Visits by Providers", interactive=False)
                prov_chart2 = gr.Image(label="Total MC by Providers", interactive=False)
            with gr.Row():
                prov_chart3 = gr.Image(label="% MC Given by Providers", interactive=False, visible=True)
            with gr.Row():
                prov_chart4 = gr.Image(label="Total Claim by Providers", interactive=False)
                prov_chart5 = gr.Image(label="Average Claim per Visit by Providers", interactive=False)
            download_btn_prov = gr.Button("Download Provider Charts as PPTX")

        with gr.TabItem("Employee Insights"):
            gr.Markdown("## Employee Insights Dashboard")
            with gr.Row():
                emp_chart1 = gr.Image(label="Total Visits by Employees", interactive=False)
                emp_chart2 = gr.Image(label="Total Claim by Employees", interactive=False)
            with gr.Row():
                emp_chart3 = gr.Image(label="Average Claim per Visit by Employees", interactive=False)
                emp_chart4 = gr.Image(label="Total MC by Employees", interactive=False)
            with gr.Row():
                emp_chart5 = gr.Image(label="Average MC per Visit by Employees", interactive=False)
                emp_chart6 = gr.Image(label="Claim Distribution by Division", interactive=False)
            download_btn_emp = gr.Button("Download Employee Charts as PPTX")

    def scrape_and_store_yearly(url, user_id, password, show_mc_pct, mc_sort_order):
        patient_data_by_year, claim_data_by_year, mc_data_by_year, status = scrape_data(url, user_id, password)
        if patient_data_by_year is None or claim_data_by_year is None or mc_data_by_year is None:
            return (
                status, None, None, None, 
                gr.update(choices=["2024"], value="2024", interactive=False),
                gr.update(choices=["Yearly"], value="Yearly", interactive=False),
                None, None, None, None, None, None, None, None, None, None, None
            )

        available_years = sorted(set(patient_data_by_year.keys()) | set(claim_data_by_year.keys()) | set(mc_data_by_year.keys()))
        year_choices = [str(year) for year in available_years]
        default_year = year_choices[0] if year_choices else "2024"
        month_choices = ["Yearly"] + [f"{year}-{month:02d}" for year in available_years for month in range(1, 13)]

        provider_images, employee_images, provider_figs, employee_figs = generate_dashboard_charts(
            patient_data_by_year, claim_data_by_year, mc_data_by_year, default_year, show_mc_pct, mc_sort_order)
        return (
            status, patient_data_by_year, claim_data_by_year, mc_data_by_year,
            gr.update(choices=year_choices, value=default_year, interactive=True),
            gr.update(choices=month_choices, value="Yearly", interactive=True),
            provider_images[0], provider_images[1], provider_images[2], provider_images[3], provider_images[4],
            employee_images[0], employee_images[1], employee_images[2], employee_images[3], employee_images[4], employee_images[5]
        )

    def scrape_and_store_monthly(url, user_id, password, show_mc_pct, mc_sort_order):
        patient_data_by_month, claim_data_by_month, mc_data_by_month, status = scrape_monthly_data(url, user_id, password)
        if patient_data_by_month is None or claim_data_by_month is None or mc_data_by_month is None:
            return (
                status, None, None, None, 
                gr.update(choices=["2024"], value="2024", interactive=False),
                gr.update(choices=["Yearly"], value="Yearly", interactive=False),
                None, None, None, None, None, None, None, None, None, None, None
            )

        available_years = sorted(set(int(k.split('-')[0]) for k in patient_data_by_month.keys()))
        year_choices = [str(year) for year in available_years]
        default_year = year_choices[0] if year_choices else "2024"
        month_choices = ["Yearly"] + sorted(patient_data_by_month.keys())

        default_period = list(patient_data_by_month.keys())[0]  # First month as default
        provider_images, employee_images, provider_figs, employee_figs = generate_dashboard_charts(
            patient_data_by_month, claim_data_by_month, mc_data_by_month, default_period, show_mc_pct, mc_sort_order)
        return (
            status, patient_data_by_month, claim_data_by_month, mc_data_by_month,
            gr.update(choices=year_choices, value=default_year, interactive=True),
            gr.update(choices=month_choices, value=default_period, interactive=True),
            provider_images[0], provider_images[1], provider_images[2], provider_images[3], provider_images[4],
            employee_images[0], employee_images[1], employee_images[2], employee_images[3], employee_images[4], employee_images[5]
        )

    def update_dashboard(year, month, patient_data, claim_data, mc_data, show_mc_pct, mc_sort_order):
        if not patient_data or not claim_data or not mc_data:
            return [None] * 11
        period = year if month == "Yearly" else month
        provider_images, employee_images, _, _ = generate_dashboard_charts(
            patient_data, claim_data, mc_data, period, show_mc_pct, mc_sort_order)
        return (
            provider_images[0], provider_images[1], provider_images[2], provider_images[3], provider_images[4],
            employee_images[0], employee_images[1], employee_images[2], employee_images[3], employee_images[4], employee_images[5]
        )

    def download_provider_pptx(year, month, patient_data, claim_data, mc_data, show_mc_pct, mc_sort_order):
        period = year if month == "Yearly" else f"{datetime.strptime(month.split('-')[1], '%m').strftime('%B')} {year}"
        _, _, provider_figs, _ = generate_dashboard_charts(
            patient_data, claim_data, mc_data, period if month == "Yearly" else month, show_mc_pct, mc_sort_order)
        pptx_path = charts_to_pptx(provider_figs, [], period)
        label = f"Provider_Charts_{period.replace(' ', '_')}.pptx"
        if pptx_path:
            return gr.File(value=pptx_path, label=label)
        else:
            print("Failed to generate provider PPTX")
            return None

    def download_employee_pptx(year, month, patient_data, claim_data, mc_data, show_mc_pct, mc_sort_order):
        period = year if month == "Yearly" else f"{datetime.strptime(month.split('-')[1], '%m').strftime('%B')} {year}"
        _, _, _, employee_figs = generate_dashboard_charts(
            patient_data, claim_data, mc_data, period if month == "Yearly" else month, show_mc_pct, mc_sort_order)
        pptx_path = charts_to_pptx([], employee_figs, period)
        label = f"Employee_Charts_{period.replace(' ', '_')}.pptx"
        if pptx_path:
            return gr.File(value=pptx_path, label=label)
        else:
            print("Failed to generate employee PPTX")
            return None

    scrape_yearly_btn.click(
        fn=scrape_and_store_yearly,
        inputs=[url_input, user_id_input, password_input, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=[
            status_output, patient_state, claim_state, mc_state, year_dropdown, month_dropdown,
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6
        ]
    )

    scrape_monthly_btn.click(
        fn=scrape_and_store_monthly,
        inputs=[url_input, user_id_input, password_input, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=[
            status_output, patient_state, claim_state, mc_state, year_dropdown, month_dropdown,
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6
        ]
    )

    year_dropdown.change(
        fn=update_dashboard,
        inputs=[year_dropdown, month_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=[
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6
        ]
    )

    month_dropdown.change(
        fn=update_dashboard,
        inputs=[year_dropdown, month_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=[
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6
        ]
    )

    show_mc_pct_checkbox.change(
        fn=update_dashboard,
        inputs=[year_dropdown, month_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=[
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6
        ]
    )

    mc_sort_dropdown.change(
        fn=update_dashboard,
        inputs=[year_dropdown, month_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=[
            prov_chart1, prov_chart2, prov_chart3, prov_chart4, prov_chart5,
            emp_chart1, emp_chart2, emp_chart3, emp_chart4, emp_chart5, emp_chart6
        ]
    )

    download_btn_prov.click(
        fn=download_provider_pptx,
        inputs=[year_dropdown, month_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=gr.File()
    )

    download_btn_emp.click(
        fn=download_employee_pptx,
        inputs=[year_dropdown, month_dropdown, patient_state, claim_state, mc_state, show_mc_pct_checkbox, mc_sort_dropdown],
        outputs=gr.File()
    )

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7868
* Running on public URL: https://74cc27668516962e75.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
